## Final 5-Year Data Assembly Script

This master script orchestrates the full pipeline across all five years, producing the final consolidated dataset. As it loops through each calendar year, you’ll see:

- **Annual GDP Trends & LOESS Decompositions**  
  We update the GDP growth inputs and rerun the LOESS smoothing for each year, exactly as described in Script 1. This ensures that seasonal patterns and economic trajectories remain tailored to that year’s specific context.

- **Satellite Imagery Integration & KNN Imputation**  
  After generating the core variables, the script imports the Sentinel-2 land-cover metrics and applies a K-Nearest Neighbors algorithm to fill any remaining missing values. This two-step process merges remote-sensing data with spatial statistics to achieve a complete, high-resolution picture.

> **Performance Note:**  
> Because this script loads multi-year Sentinel-2 collections and executes memory-intensive KNN operations, it can easily consume tens of gigabytes of RAM when run end-to-end. To improve stability and speed:
> 1. **Process by Year:** Break the script into individual year modules and run them sequentially.  
> 2. **Use Cloud or Parallel Compute:** Leverage Google Earth Engine batch exports or parallel Python workers to distribute the load.  
> 3. **Chunk & Cache Data:** Cache intermediate results (e.g., annual LOESS outputs, satellite masks) to disk or cloud storage to avoid repeated recalculations.  

By following these best practices, you’ll maintain reproducibility while keeping resource usage within practical limits.


In [2]:
import pandas as pd
import re
import unicodedata
import geopandas as gpd
from shapely.geometry import Point
import time 
import matplotlib.pyplot as plt
from pykrige.ok import OrdinaryKriging
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from geopy.distance import geodesic
from pykrige.kriging_tools import write_asc_grid
from sklearn.preprocessing import MinMaxScaler

#Decoradores utiles

def decorador_2(func):
    def wrapped(*args):
        time1 = time.time()
        rta = func(*args)
        time2 = time.time()
        print('La funcion demoro' , time2-time1)
        return rta 
    return wrapped 


# Funciones que vamos a utilizar en todo el script 

def eliminar_tildes(str):
    return ''.join(
        i for i in unicodedata.normalize('NFKD', str)
        if unicodedata.category(i) != 'Mn')

meses = {
    1: "January", 2: "February", 3: "March", 4: "April", 
    5: "May", 6: "June", 7: "July", 8: "August", 
    9: "September", 10: "October", 11: "November", 12: "December"
}

meses_esp = {
    "Enero": "January", "Febrero": "February", "Marzo": "March", "Abril": "April", 
    "Mayo": "May", "Junio": "June", "Julio": "July", "Agosto": "August", 
    "Septiembre": "September", "Octubre": "October", "Noviembre": "November", "Diciembre": "December"
}

@decorador_2
def arreglar_texto(df , variable , nombre):
    if variable not in df.columns:
        print('La columna' , variable ,'no existe en el DataFrame')
        return df
        
    df[variable] = df[variable].str.lower()
    df[variable] = df[variable].str.replace(' ', '', regex=False)
    df[variable] = df[variable].str.replace(r'[^\w\s]', '', regex=True)   
    df[variable] = df[variable].str.replace('sanandresdetumaco', 'tumaco', regex=False)
    df[variable] = df[variable].str.replace('ct$', '', regex=True)
    df[variable] = df[variable].str.replace('bogotadc', 'bogota', regex=False)
    df[variable] = df[variable].apply(eliminar_tildes)
    df = df.rename(columns={variable: nombre})
    return df

@decorador_2
def completar_meses(df , escala , mes , observaciones , operacion=0 ):
    if operacion ==0:
        #Hacer un groupby sacando la mediana de las observaciones por mes
        df = df.groupby([escala, mes])[observaciones].median().reset_index()
        municipios = df[escala].unique()
        meses1 = range(1, 13)
        df_combinaciones = pd.MultiIndex.from_product([municipios, meses1], names=[escala, mes]).to_frame(index=False)
        df_combinaciones[mes] = df_combinaciones[mes].map(meses)
        Final= pd.merge(df_combinaciones, df, on=[escala, mes], how='left')
        return Final
    else:
        #Hacer un groupby sumando las observaciones por mes
        conteo = df.groupby([escala, mes])[observaciones].sum().reset_index()
        municipios = df[escala].unique()
        meses1 = range(1, 13)
        df_combinaciones = pd.MultiIndex.from_product([municipios, meses1], names=[escala, mes ]).to_frame(index=False)
        df_completo = pd.merge(df_combinaciones, conteo, on=[escala, mes], how='left')
        df_completo[observaciones] = df_completo[observaciones].fillna(0)
        df_completo[mes] = df_completo[mes].map(meses)
        return df_completo 
    

def meses_a_numeros(df , mes):
    meses_invertido = {i: k for k, i in meses.items()}
    df[mes] = df[mes].map(meses_invertido)
    return df

def traducir(df , mes ):
    df[mes] = df[mes].map(meses_esp)
    return df

@decorador_2
def bases_crimenes(df , dia , cantidad , nombre):
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    df[dia] = pd.to_datetime(df[dia])
    df['Mes'] = df[dia].dt.month_name()
    df = arreglar_texto(df , 'MUNICIPIO', 'Ciudad')
    df1= df.groupby(['Ciudad' , 'Mes'])[cantidad].sum().reset_index()
    df1 = df1.rename(columns={cantidad: nombre})
    return df1
    
def convertir_ubicacion(df , variable):
    df['ubicacion'] = df[variable].str.strip('()')
    df[['latitud', 'longitud']] = df['ubicacion'].str.split(',', expand=True)
    df['latitud'] = df['latitud'].astype(float)
    df['longitud'] = df['longitud'].astype(float)
    df['geometry'] = df.apply(lambda row: Point(row['longitud'], row['latitud']), axis=1)
    df = df.drop([variable , 'ubicacion' , 'latitud' , 'longitud'] , axis=1)
    return df

Top_Colombia = {'Ciudad Amurallada' : [(10.423768016601226, -75.5502221481417) , 23267+1851 , 4.8] , 
                'Monserrate':[(4.606442657411985, -74.05488182882448) , 16061+10034 , 4.7],    
                'Museo Oro': [(4.601863737486862, -74.07206921955388) , 14606+43837 , 4.8], 
                'Museo Pablo Escobar':[(6.210940942225721, -75.55666610034157) , 4041+2460 , 3.8],
                'Castillo San Felipe': [(10.422674818711775, -75.53911007123797) , 13188+61752 , 4.7],
                'La Candelaria': [(4.596356204217971, -74.07036887708271), 4790+35201 , 4.6 ],
                'Museo Botero': [(4.596783977356232, -74.07317983196756) , 10511+21480 , 4.8],
                'Piedra Penol': [(6.224293398570674, -75.17846180492857) , 2537+3722 , 4.7],
                'Parque Tayrona':[(11.30661973931487, -74.06558444232788) , 6407+24124 , 4.7],
                'Ciudad Perdida': [(10.69977509707966, -74.21719016507225) , 2129+10293 , 4.5],
                'Johnny Cay': [(12.599844555163797, -81.68958552452854) , 6944+1193, 4.6 ],
                'Mina sal': [(5.064130673191533, -73.87412723129202) , 4946+10681 , 4.8],
                'Parque Explora': [(6.271873980048277, -75.56529352888349), 3290+6868 , 4.8],
                'Parque del cafe': [(4.540472412992844, -75.7704120656708) , 2260+64813 , 4.8],
                'Getsemani': [(10.420236347859726, -75.5457920284369) , 827+9890 , 4.7],
                'Laguna Guatavita': [(4.978042131016031, -73.77411934630805) , 1641+760 , 4.6],
                'Laguna Guatape': [(6.2625426509484585, -75.18889681059652) , 1667+1920 , 4.9],
                'Zoo Cali': [(3.4483110753559307, -76.5567835595829) , 2505+30375 , 4.7],
                'Jaime Duque':[(4.950933448804722, -73.96450197491808) , 608+5558 , 4.8],
                'Museo Antioquia':[(6.252582668979805, -75.56902381539014) , 2037+8372 , 4.7],
                'Bosque Palmas': [(4.642619785055463, -75.48545072059876) , 1999+6026 , 4.8],
                'santuario san pedro claver' : [(10.421942528179812, -75.55101547480216) , 2344+2868 , 4.7],
                'Santuario Las Lajas': [(0.8056145166329276, -77.58580465582233) , 868+10768 , 4.8],
                'Medellin MetroCable': [(6.293713286896887, -75.54049102937564) , 2505+6768 , 4.7],
                'Villa de leyva': [(5.636740186520528, -73.52642769258925) , 10523+2931 , 4.8],
                'Salento': [(4.637632624472757, -75.57107751600738) , 935+1391 , 4,7 ],
                'Islas Rosario': [(10.198113718483176, -75.75728109040843) , 1293+235 , 4.5],
                'Cano cristales' : [(2.2654672947309984, -73.7904893534562) , 442+303 , 4.9],
                'Cabo de la vela': [(12.19584280316732, -72.14670358044717) , 503 , 4.7],
                'Parque Nacional Natural El Cocuy' :[(6.366594015120004, -72.32930605910018) , 1468,4.8],
                'Jardin': [(5.599970208507919, -75.82197889439527) , 1923 , 4.7],
                'Canon guejar': [(3.34558725490291, -74.00251320801074) , 223 , 5],
                'Chingaza': [(4.736759401563025, -73.84183421964912) , 312+532 , 4.9],
                'Selva Amazonas': [(-1.9373373758415269, -70.09301207031643) , 88 , 4.5],
                'Rio miel': [(5.77684008762934, -74.68539501381262) , 42+33 , 4.2],
                'Cerros de Mavecure': [(3.8685506255477473, -67.90980077915421) , 47+122 , 4.7],
                'Desiero Tatacoa': [(3.233661252748834, -75.167678435162) , 844+2805 , 4.8],
                'Parque Uramba': [(4.009359201162415, -77.24025001985369) , 51+2034 , 4.7],
                'Naqui': [(5.711911987713407, -77.26317845045835) , 1239 , 4.6],
                'San Jose Guaviare':[(2.5680632175041964, -72.6400738137754) , 4312 , 4.8],
                'Cascada Fin del Mundo':[(1.1049043406387264, -76.61366052942371) , 138 , 4.8],
                    }


# Import heavy databases first

In [3]:
clima = pd.read_csv("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241001.csv")

C:\Users\alejo\AppData\Local\Temp\ipykernel_30804\1820426153.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  clima = pd.read_csv("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241001.csv")


In [4]:

clima['FechaObservacion'] = pd.to_datetime(clima['FechaObservacion'])
clima['Mes'] = clima['FechaObservacion'].dt.month_name()


C:\Users\alejo\AppData\Local\Temp\ipykernel_30804\429284614.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clima['FechaObservacion'] = pd.to_datetime(clima['FechaObservacion'])


# 2018

In [ ]:
#Inicio proceso
df= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Visitantes_No_Residentes.csv')
df1 = df[df["Año"] == 2018]
df1= arreglar_texto(df1 , 'Ciudad' , 'Ciudad')
Homicidios= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\homicidios_2018_1.xlsx').dropna()
Homicidos1 = bases_crimenes(Homicidios , 'FECHA HECHO' , 'CANTIDAD' , 'Homicidios')
Hurto= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\hurto_a_personas_2018_1.xlsx').dropna()
Hurto1 = bases_crimenes(Hurto , 'FECHA HECHO' , 'CANTIDAD' , 'Hurtos')
sexuales= sexuales= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\delitos_sexuales_2018_0.xlsx').dropna()
sexuales1 = bases_crimenes(sexuales , 'FECHA HECHO' , 'CANTIDAD' , 'Delitos Sexuales')
ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades = arreglar_texto(ciudades, 'city' , "Ciudad")
ciudades['admin_name'] = ciudades['admin_name'].astype(str)
ciudades.at[68, 'admin_name'] = 'Atlantico'
Total_18= completar_meses(df1 , 'Ciudad' , 'Mes' ,'Extranjeros no Residentes' , 1)
Total_18 = pd.merge(Total_18, Homicidos1[['Ciudad', 'Homicidios' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, Hurto1[['Ciudad', 'Hurtos' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, sexuales1[['Ciudad', 'Delitos Sexuales' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18['Homicidios'] = Total_18['Homicidios'].fillna(0)
Total_18['Hurtos'] = Total_18['Hurtos'].fillna(0)
Total_18['Delitos Sexuales'] = Total_18['Delitos Sexuales'].fillna(0)
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('bogotadc', 'bogota', regex=False)

#Seleccionar Ciudades que vamos a trabajar
ciudades1= set(ciudades['Ciudad'])
ciudades2= set(Total_18['Ciudad'])
grandes_ciudades=list(ciudades1.intersection(ciudades2))
l=['bogota' , 'tumaco']
grandes_ciudades = grandes_ciudades + l
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('sanandresdetumaco', 'tumaco', regex=False)

for i in Total_18['Ciudad']:
    indice = Total_18[Total_18['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Total_18 = Total_18.drop(index=indice)
    else:
        pass

#Poner Ubicaciones
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades_geometry= gpd.GeoDataFrame(ciudades, geometry='geometry')
Total_18_Geometry= pd.merge(Total_18, ciudades_geometry[['Ciudad', 'geometry']], on='Ciudad' , how='left')
Total_18_Geometry= gpd.GeoDataFrame(Total_18_Geometry, geometry='geometry')

# Poner departamentos
Departamentos = gpd.read_file("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Colombia.json")
Departamentos = arreglar_texto(Departamentos, 'NOMBRE_DPT' , "Departamento")
Departamentos["Departamento"] = Departamentos["Departamento"].str.replace('santafedebogota', 'bogotadc', regex=False)

if Total_18_Geometry.crs is None:
    Total_18_Geometry = Total_18_Geometry.set_crs(Departamentos.crs, allow_override=True)

Total_18_Geometry = gpd.sjoin_nearest(Total_18_Geometry,  Departamentos, how='left', distance_col='distancia').drop(['AREA' , 'PERIMETER' , 'HECTARES' , 'DPTO','index_right' , 'distancia'] , axis=1)
Total_18_Geometry
Total_18_Geometry[Total_18_Geometry.isnull().any(axis=1)]

#Agregar Clima
clima_2018 = clima[clima['FechaObservacion'].dt.year == 2018]
clima_2018 = arreglar_texto(clima_2018 , 'Municipio', 'Ciudad')
clima_2018= completar_meses(clima_2018 , 'Ciudad' , 'Mes' ,'ValorObservado' , 0)
Total_18_Clima= pd.merge(Total_18_Geometry, clima_2018[['Ciudad', 'ValorObservado' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18_Clima = Total_18_Clima.rename(columns={'ValorObservado': 'Temperatura'})

#Krigging para el clima 
df_kriging = meses_a_numeros(Total_18_Clima , 'Mes')
df_kriging.set_crs(epsg=4326, inplace=True)
df_kriging=df_kriging.to_crs(epsg=32618)  

meses_krig = df_kriging['Mes'].unique()

for i in meses_krig:
    print('Procesando Mes:' ,i)
    
    df_kriging_mes = df_kriging[df_kriging['Mes'] == i] # Vamos a realizar el kriging por mes
    df_kriging_known = df_kriging_mes.dropna(subset=['Temperatura']) # Los datos para los que tenemos temperatura
    df_kriging_missing = df_kriging_mes[df_kriging_mes ['Temperatura'].isna()] # Los datos para los NO que tenemos temperatura (Variable a predecir)
    
    if df_kriging_missing.empty:
        print('No hay datos faltantes en el mes ' , i) # Si algun mes esta completo para todas las ciudades, no hacer nada 
        continue
    
    # Nuestras variables para predecir
    x_known = df_kriging_known.geometry.x.values
    y_known = df_kriging_known.geometry.y.values
    z_known = df_kriging_known['Temperatura'].values
    
    # Nuestros datos a predecir
    x_missing = df_kriging_missing.geometry.x.values
    y_missing = df_kriging_missing.geometry.y.values
    
    # Crear el modelo 
    Krigg = OrdinaryKriging(
        x_known, y_known, z_known,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False)
    
    # Realizar predicciones con el krigging
    z_pred, ss = Krigg.execute('points', x_missing, y_missing)
    
    # Introducir en el DF los valores
    df_kriging.loc[df_kriging_missing.index, 'Temperatura'] = z_pred

#Agregar precio dolar
Dolar = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Dolar.xlsx')
Dolar['Fecha'] = pd.to_datetime(Dolar['Fecha'], errors='coerce')
Dolar.set_index('Fecha', inplace=True)
Dolar = Dolar.resample('M').median()
Dolar = Dolar.reset_index()
Dolar_2018=Dolar[Dolar['Fecha'].dt.year == 2018]
Dolar_2018['Mes'] = range(1, 13)
Base_2018=pd.merge(df_kriging, Dolar_2018, on= 'Mes', how='right').drop('Fecha' , axis=1)
Base_2018

#Interpolar PIB mensual

PIB= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\PIB - Miles de millones de pesos  - 2018.xlsx')
PIB= arreglar_texto(PIB, 'DEPARTAMENTO' , 'Departamento')
PIB['VALOR (unidades)'] = PIB['VALOR (unidades)'].str.replace(',', '.')
PIB['VALOR (unidades)'] = pd.to_numeric(PIB['VALOR (unidades)'], errors='coerce')

# Usaremos el metodo de LOESS para descomponer el PIB mensualmente 
meses_loees= np.arange(1, 13)
tendencia_mensual_inflacion = [3.68 ,3.37, 3.14 , 3.13 , 3.16 , 3.20 , 3.12 , 3.10 , 3.23 , 3.33 , 3.27 , 3.18] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual_inflacion) for valor in tendencia_mensual_inflacion]) 
patron_estacional = patron_estacional / patron_estacional.sum()

# La idea es crear algun tipo de ruido sobre nuestro patron, para asi tratar de simular efectos reales economicos.
np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) #  Asegurarnos que todos las fulctuaciones sean positivas. 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()


def descomponer_pib_loess(pib_anual, patron):
    def asignar_pib_mensual(k, patron):
        pib_mensual = patron * k['VALOR (unidades)']
        return pib_mensual

    df_mensual = pd.DataFrame()
    
    for i, k in pib_anual.iterrows():
        pib_mensual = asignar_pib_mensual(k, patron)
        
        df_temp = pd.DataFrame({
            'Departamento': k['Departamento'],
            'Mes': np.arange(1, 13),
            'PIB_Mensual': pib_mensual
        })
        
        df_mensual = pd.concat([df_mensual, df_temp], ignore_index=True)
    
    return df_mensual

pib_mensual = descomponer_pib_loess(PIB, patron_smooth)
importancia_pesos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\importancia-municipal.xlsx')

#Dado que encontrar una serie completa desde 2018 hasta 2024 de el valor del PIB por ciudad no es facil, primero usaremos la descomposicon por mes que hicimos departamental y la ponderaremos 
#por la importancia de cada ciudad en el departamento para encontrar un aproximando significativo. pdta= la base que nos pasaste profe no tenia todos los anos por eso hacemos esta aproximacion
importancia_pesos = arreglar_texto(importancia_pesos , 'Municipio / Distrito' , 'Ciudad')
importancia_pesos['Departamento'] = importancia_pesos['Departamento'].astype(str)
importancia_pesos = arreglar_texto(importancia_pesos , 'Departamento' , 'Departamento')

# Dejar solo las ciudades que estamos trabajando.
for i in importancia_pesos['Ciudad']:
    indice = importancia_pesos[importancia_pesos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        importancia_pesos = importancia_pesos.drop(index=indice)
    else:
        pass

importancia_pesos_mensual = importancia_pesos.loc[importancia_pesos.index.repeat(12)].reset_index(drop=True)
importancia_pesos_mensual['Mes'] = (importancia_pesos_mensual.index % 12) + 1
pib_pesos_ciudades1 = pd.merge(importancia_pesos_mensual, pib_mensual, on=['Departamento', 'Mes'], how='left')
pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] /100
pib_pesos_ciudades1['Pib Ponderado'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] * pib_pesos_ciudades1['PIB_Mensual']
fusion_left = pd.merge(Base_2018, pib_pesos_ciudades1[['Pib Ponderado' , 'Ciudad', 'Mes']], on=['Ciudad', 'Mes'], how='left')

#Agregar puntos de llegadas internacionales
entradas = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Entradas_de_extranjeros_a_Colombia_20241006.csv')
entradas_2018= entradas[entradas['Año']==2018]
entradas_2018 = entradas_2018[entradas_2018['Latitud - Longitud'] != 'No Aplica,No Aplica']
entradas_2018 =traducir(entradas_2018 , 'Mes')
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )
entradas_2018 =completar_meses(entradas_2018, 'Latitud - Longitud' , 'Mes' , 'Total' ,1 )
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )

entradas_geometry = convertir_ubicacion(entradas_2018 , 'Latitud - Longitud')
Entradas = gpd.GeoDataFrame(entradas_geometry, geometry='geometry')

if Entradas.crs is None:
    Entradas = Entradas.set_crs(Departamentos.crs, allow_override=True)

Entradas = gpd.sjoin_nearest(Entradas,  Departamentos, how='left', distance_col='distancia').drop(['index_right' , 'DPTO' , 'AREA' , 'PERIMETER' , 'HECTARES' ,'distancia'] , axis=1)
Entradas_Departamentos = Entradas.groupby(['Mes', 'Departamento'])['Total'].sum().reset_index()
final = pd.merge(fusion_left, Entradas_Departamentos, on=['Departamento', 'Mes'], how='left').rename(columns={'Total': 'Entradas Extranjeros Zona'})
final['Entradas Extranjeros Zona'] = final['Entradas Extranjeros Zona'].fillna(0)

#Distancias desde los puntos de llegadas internacionales ponderadas 
for i in ciudades['Ciudad']:
    indice = ciudades[ciudades['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_unico = ciudades.drop(index=indice)
    else:
        pass

ciudades_unico = ciudades_unico.iloc[:, [0, -1]]
ciudades_unico= gpd.GeoDataFrame(ciudades_unico, geometry='geometry')
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')

if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if Migracion_Unico.crs is None:
    Migracion_Unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de Migracion_Unico: {Migracion_Unico.crs}")

ciudades_unico = ciudades_unico.to_crs(epsg=32618)
Migracion_Unico = Migracion_Unico.to_crs(epsg=32618)

ciudades_unico['key'] = 1
Migracion_Unico['key'] = 1

Distancia_combinado = pd.merge(ciudades_unico, Migracion_Unico, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

Distancia_combinado['distancia_km'] = Distancia_combinado.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_combinado['distancia_ponderada'] = Distancia_combinado['distancia_km'] * Distancia_combinado['Total']
Distancia_ponderado = Distancia_combinado.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_migrantes=('Total', 'sum')
).reset_index()

Distancia_ponderado['distancia_ponderada_km'] = Distancia_ponderado['suma_ponderada'] / Distancia_ponderado['suma_migrantes']
Distancia_ponderado_final = Distancia_ponderado[['Ciudad' , 'distancia_ponderada_km']]

Distancia_ciudades = ciudades_unico.merge(Distancia_ponderado_final, on='Ciudad')
final1 = pd.merge(final, Distancia_ciudades, on='Ciudad', how='left').drop(['geometry_y' , 'key'] , axis=1)\


#Score de la importancia de los aeropuertos
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')
columnas = ['geometry' , 'Departamento' ,'Total' ]
Migracion_Unico_Dept = gpd.sjoin_nearest(Migracion_Unico, Departamentos, how='left', distance_col='distancia')[columnas]
Total_Migracion = Migracion_Unico_Dept.groupby('Departamento').agg(
    puntos=('Departamento', 'size'),  
    total_personas=('Total', 'sum')  
).reset_index()
scaler = MinMaxScaler()
Total_Migracion[['puntos_norm', 'total_norm']] = scaler.fit_transform(Total_Migracion[['puntos', 'total_personas']])
Total_Migracion['importancia accesos'] = 0.3 * Total_Migracion['puntos_norm'] + 0.7 * Total_Migracion['total_norm'] # Asumimos que es mas importante cuanta gente llega a que tantos puntos hay.
final1 = pd.merge(final1, Total_Migracion, on='Departamento', how='left').drop(['puntos' , 'total_personas' , 'puntos_norm' , 'total_norm'] , axis=1)
final1['importancia accesos'] = final1['importancia accesos'].fillna(0)

#Numero establecimientos de turismo, camas y habitaciones para turistas.
hoteles = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Hist_rico_Registro_Nacional_de_Turismo_-_RNT_20241007.csv')
establecimientos_2018 = hoteles[hoteles['AÑO']==2018]

Establecimientos = establecimientos_2018.groupby('NOMBRE-MUNI').agg({'CATEGORIA': 'count','HABITACIONES': 'sum', 'CAMAS': 'sum'}).reset_index().rename(columns={'CATEGORIA': 'Establecimientos'})
Establecimientos = arreglar_texto(Establecimientos, 'NOMBRE-MUNI' , 'Municipios')
for i in Establecimientos['Municipios']:
    indice = Establecimientos[Establecimientos['Municipios'] == i].index
    if i not in grandes_ciudades:
        Establecimientos = Establecimientos.drop(index=indice)
    else:
        pass

Establecimientos=Establecimientos.rename(columns={'Municipios': 'Ciudad'})
meses_loees = np.arange(1, 13)
patron_estacional = np.array([1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12]) 
patron_estacional = patron_estacional / patron_estacional.sum()

np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()

def descomponer_variables_loess(df_anual, patron, variables):
    def asignar_valores_mensuales(fila, patron, variables):
        valores_mensuales = {}
        for i in variables:
            valores_mensuales[i] = patron * fila[i]
        return valores_mensuales

    data_mensual = []
    for i, fila in df_anual.iterrows():
        valores_mensuales = asignar_valores_mensuales(fila, patron, variables)
        
        df_temp = pd.DataFrame({
            'Ciudad': fila['Ciudad'],
            'Mes': np.arange(1, 13)
        })
        
        for k in variables:
            df_temp[k] = valores_mensuales[k]
        
        data_mensual.append(df_temp)
    
    df_mensual = pd.concat(data_mensual, ignore_index=True)
    
    return df_mensual

Establecimientos_mensual = descomponer_variables_loess(Establecimientos, patron_smooth , ['Establecimientos' , 'HABITACIONES' , 'CAMAS'])
final2 = pd.merge(final1, Establecimientos_mensual, on=['Ciudad', 'Mes'], how='left')

#Distancias al TOP de colombia 

df_TopColombia = pd.DataFrame.from_dict(Top_Colombia, orient='index', columns=['coordinates','Reviews', 'rating', 'eliminar'])
df_TopColombia = df_TopColombia.reset_index().rename(columns={'index': 'lugar'}).drop('eliminar' , axis=1)
df_TopColombia['geometry'] = df_TopColombia['coordinates'].apply(lambda x: Point(x[1], x[0]))
df_TopColombia_gdp = gpd.GeoDataFrame(df_TopColombia, geometry='geometry').drop('coordinates' , axis=1)
if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if df_TopColombia_gdp.crs is None:
    df_TopColombia_gdp.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de df_TopColombia_gdp: {df_TopColombia_gdp.crs}")


ciudades_unico = ciudades_unico.to_crs(epsg=32618)
df_TopColombia_gdp = df_TopColombia_gdp.to_crs(epsg=32618)

scaler = MinMaxScaler()

ciudades_unico['key'] = 1
df_TopColombia_gdp['key'] = 1

Distancia_Top = pd.merge(ciudades_unico, df_TopColombia_gdp, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

w_reviews = 0.6 # Vamos a asumir que las reviews tienen mas peso que la calificacion del lugar 
w_rating = 0.4

Distancia_Top['distancia_km'] = Distancia_Top.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_Top[['distancia_km','Reviews', 'rating']] = scaler.fit_transform(
    Distancia_Top[['distancia_km','Reviews', 'rating']]
) # Normalizamos las variables para calcular una distancia ponderada con las mismas escalas

Distancia_Top['distancia_ponderada'] = (Distancia_Top['distancia_km']) + ( w_reviews* Distancia_Top['Reviews']) + ( w_rating* Distancia_Top['rating'])

Distancia_Ponderado_Top = Distancia_Top.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_reviews=('Reviews', 'sum'),
    suma_rating=('rating', 'sum')).reset_index()

Distancia_Ponderado_Top['distancia_ponderada_TOP'] = Distancia_Ponderado_Top['suma_ponderada'] / (Distancia_Ponderado_Top['suma_reviews']+Distancia_Ponderado_Top['suma_rating'])
Distancia_Ponderado_Top_final = Distancia_Ponderado_Top[['Ciudad' , 'distancia_ponderada_TOP']]

Distancia_Top = ciudades_unico.merge(Distancia_Ponderado_Top_final, on='Ciudad')
final3 = pd.merge(final2, Distancia_Top, on='Ciudad', how='left').drop(['geometry_x' , 'key'] , axis=1)

# Pobreza con proxy
ciudades_poblacion = ciudades[['Ciudad' , 'population']]
for i in ciudades_poblacion['Ciudad']:
    indice = ciudades_poblacion[ciudades_poblacion['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_poblacion = ciudades_poblacion.drop(index=indice)
    else:
        pass

final4 = pd.merge(final3, ciudades_poblacion, on='Ciudad', how='left')
final4['Proxy_Pobreza'] = final4['Pib Ponderado'] / final4['population']
final4

#Gasto promedio por dia de un turista
gastos_diarios = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Gasto Promedio Diario 2019.xlsx')
gastos_diarios = arreglar_texto(gastos_diarios , 'Ciudad', 'Ciudad')
ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_diarios = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='left')

inflacion_2019_promedio = 3.80 / 100
inflacion_2019_aliemntos= 5.80 / 100
inflacion_2019_transporte= 3.41 /100
inflacion_2019_alojamiento = 3.46/100

l2=[inflacion_2019_promedio, inflacion_2019_alojamiento,inflacion_2019_transporte,inflacion_2019_aliemntos,inflacion_2019_promedio]
c=-1
for i in gastos_diarios.iloc[: ,1:-1]:
    c=c+1
    gastos_diarios[i] = gastos_diarios[i] / (1+ l2[c])


gastos_diarios_Nan = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_diarios_Nan = gpd.GeoDataFrame(pd.merge(gastos_diarios_Nan, ciudades_geometry, on='Ciudad', how='left').drop('geometry_x' , axis=1) , geometry='geometry')
gastos_diarios_Nan.set_crs(epsg=4326, inplace=True)
gastos_diarios_Nan=gastos_diarios_Nan.to_crs(epsg=32618)  
variables = [i for i in gastos_diarios_Nan.columns][1:-1]
gastos_sin_Nan = gastos_diarios_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_diarios_Nan[gastos_diarios_Nan[variables].isnull().any(axis=1)]

def kriging(df_con_info, variable, grid_size=100):
    coords = np.array(list(zip(df_con_info.geometry.x, df_con_info.geometry.y)))
    valores = df_con_info[variable].values
    
    min_x, min_y, max_x, max_y = gastos_diarios_Nan.total_bounds
    gridx = np.linspace(min_x, max_x, grid_size)
    gridy = np.linspace(min_y, max_y, grid_size)
    
    Krigg_O = OrdinaryKriging(
        coords[:,0], coords[:,1], valores,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False
    )
    
    z1, ss1 = Krigg_O.execute('grid', gridx, gridy)
    
    return Krigg_O, gridx, gridy, z1

def estimar(Krigg, puntos):
    x = puntos.geometry.x.values
    y = puntos.geometry.y.values
    estimados, ss = Krigg.execute('points', x, y)
    return estimados

for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual['Mes'] = (gastos_krigg_mensual.index % 12) + 1
final5 = pd.merge(final4, gastos_krigg_mensual, on=['Mes', 'Ciudad'], how='left').drop('population' , axis=1)

# Gastos de todo el viaje 
gastos_viaje = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Gasto Promedio Viaje 2019.xlsx')
gastos_viaje = arreglar_texto(gastos_viaje , 'Ciudad', 'Ciudad')

ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_viaje = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='left')


l2=[inflacion_2019_promedio, inflacion_2019_alojamiento,inflacion_2019_transporte,inflacion_2019_aliemntos,inflacion_2019_promedio]
c=-1
for i in gastos_viaje.iloc[: ,1:-1]:
    c=c+1
    gastos_viaje[i] = gastos_viaje[i] / (1+ l2[c])

gastos_viaje_Nan = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_viaje_Nan = gpd.GeoDataFrame(pd.merge(gastos_viaje_Nan, ciudades_geometry, on='Ciudad', how='left') , geometry='geometry')
gastos_viaje_Nan = gastos_viaje_Nan.drop('geometry_x' , axis=1)

gastos_viaje_Nan.set_crs(epsg=4326, inplace=True)
gastos_viaje_Nan=gastos_viaje_Nan.to_crs(epsg=32618) 
variables = [i for i in gastos_viaje_Nan.columns][1:-1]
gastos_sin_Nan = gastos_viaje_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_viaje_Nan[gastos_viaje_Nan[variables].isnull().any(axis=1)]
for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual_viaje = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual_viaje['Mes'] = (gastos_krigg_mensual_viaje.index % 12) + 1
gastos_krigg_mensual_viaje
Base_2018_Final = pd.merge(final5, gastos_krigg_mensual_viaje, on=['Mes', 'Ciudad'], how='left').drop('geometry_y' , axis=1)

# Agregar inflacion 
inflacion = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Inflacion.xlsx')
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion[inflacion['Fecha'].dt.year == 2018].rename(columns={'Fecha':'Mes'})
inflacion['Mes'] = inflacion['Mes'].dt.month
Base_2018_Final = pd.merge(Base_2018_Final, inflacion, on='Mes', how='left')

#Agregar Vias
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2018_Final = pd.merge(Base_2018_Final , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

# Agregar los eventos
eventos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Eventos Turisticos\\Eventos 2018.xlsx').fillna(0)
eventos = arreglar_texto(eventos ,'Departamento' , 'Departamento')
eventos['Departamento'] = eventos['Departamento'].str.replace('sanandres', 'archipielagodesanandresprovidenciaysantacatalina', regex=False)
Base_2018_Final = pd.merge(Base_2018_Final , eventos , on=['Departamento' ,'Mes'] , how='left')

## Lets see 2018 DataFrame 

In [11]:
Base_2018_Final

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos
0,pitalito,1,33.0,2,61,15,huila,18.055550,2855.86,151.999238,...,22348.735798,338814.831038,51427.415415,83910.734197,86719.142328,129164.039202,POINT (384119.441 209921.192),3.679528,6,0.0
1,riohacha,1,108.0,4,49,8,laguajira,26.700000,2855.86,245.473018,...,25608.302505,407012.594412,44746.452735,105537.118267,85981.834594,169633.602119,POINT (728275.629 1276987.69),3.679528,2,0.0
2,rionegro,1,271.0,2,51,7,antioquia,15.500000,2855.86,343.717029,...,23317.813823,300105.482181,51759.383143,56151.273165,86719.142328,88352.122091,POINT (458587.876 680187.339),3.679528,12,3.0
3,bogota,1,99767.0,83,7149,426,bogotadc,12.600000,2855.86,23953.819812,...,33676.461464,459819.807322,95874.495457,75289.853012,116471.723062,170536.499037,POINT (602898.9 520798.299),3.679528,13,0.0
4,piedecuesta,1,28.0,1,66,13,santander,22.523648,2855.86,258.755835,...,23450.014915,316598.313028,59438.480482,63676.087723,86267.558458,96363.989201,POINT (720906.443 783432.99),3.679528,4,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,aguachica,12,14.0,2,28,4,cesar,27.550000,3198.45,73.597907,...,24460.519917,333762.080665,59294.963216,67266.228511,86719.142328,110529.693825,POINT (650504.481 919569.475),3.178001,13,0.0
968,sanjosedelguaviare,12,8.0,1,12,11,guaviare,21.841149,3198.45,38.317074,...,27668.564574,395447.494787,45450.965582,88726.234789,86719.142328,192305.361801,POINT (763167.066 283942.885),3.178001,2,0.0
969,palermo,12,1.0,0,8,0,huila,21.841149,3198.45,48.387369,...,22631.547867,307163.189741,38876.115932,67538.785887,86615.106081,103876.913144,POINT (451378.305 319631.823),3.178001,2,0.0
970,mitu,12,0.0,0,0,4,vaupes,21.841149,3198.45,14.093870,...,28137.854413,399306.017324,41051.943230,79640.264317,86719.142328,208543.305296,POINT (1037614.182 132922.769),3.178001,0,0.0


In [12]:
Base_2018_Final[Base_2018_Final.isna().any(axis=1)]

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos


# 2019

In [ ]:
df= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Visitantes_No_Residentes.csv')
df1 = df[df["Año"] == 2019]
df1= arreglar_texto(df1 , 'Ciudad' , 'Ciudad')
Homicidios= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2019\\homicidios_2019_3.xlsx').dropna()
Homicidos1 = bases_crimenes(Homicidios , 'FECHA HECHO' , 'CANTIDAD' , 'Homicidios')
Hurto= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2019\\hurto_a_personas_2019_0.xlsx').dropna()
Hurto1 = bases_crimenes(Hurto , 'FECHA HECHO' , 'CANTIDAD' , 'Hurtos')
sexuales= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2019\\delitos_sexuales_2019_0.xlsx').dropna()
sexuales1 = bases_crimenes(sexuales , 'FECHA HECHO' , 'CANTIDAD' , 'Delitos Sexuales')
ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades = arreglar_texto(ciudades, 'city' , "Ciudad")
ciudades['admin_name'] = ciudades['admin_name'].astype(str)
ciudades.at[68, 'admin_name'] = 'Atlantico'
Total_18= completar_meses(df1 , 'Ciudad' , 'Mes' ,'Extranjeros no Residentes' , 1)
Total_18 = pd.merge(Total_18, Homicidos1[['Ciudad', 'Homicidios' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, Hurto1[['Ciudad', 'Hurtos' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, sexuales1[['Ciudad', 'Delitos Sexuales' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18['Homicidios'] = Total_18['Homicidios'].fillna(0)
Total_18['Hurtos'] = Total_18['Hurtos'].fillna(0)
Total_18['Delitos Sexuales'] = Total_18['Delitos Sexuales'].fillna(0)
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('bogotadc', 'bogota', regex=False)

#Seleccionar Ciudades que vamos a trabajar
ciudades1= set(ciudades['Ciudad'])
ciudades2= set(Total_18['Ciudad'])
grandes_ciudades=list(ciudades1.intersection(ciudades2))
l=['bogota' , 'tumaco']
grandes_ciudades = grandes_ciudades + l
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('sanandresdetumaco', 'tumaco', regex=False)
for i in Total_18['Ciudad']:
    indice = Total_18[Total_18['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Total_18 = Total_18.drop(index=indice)
    else:
        pass

#Poner Ubicaciones
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades_geometry= gpd.GeoDataFrame(ciudades, geometry='geometry')
Total_18_Geometry= pd.merge(Total_18, ciudades_geometry[['Ciudad', 'geometry']], on='Ciudad' , how='left')
Total_18_Geometry= gpd.GeoDataFrame(Total_18_Geometry, geometry='geometry')

# Poner departamentos
Departamentos = gpd.read_file("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Colombia.json")
Departamentos = arreglar_texto(Departamentos, 'NOMBRE_DPT' , "Departamento")
Departamentos["Departamento"] = Departamentos["Departamento"].str.replace('santafedebogota', 'bogotadc', regex=False)

if Total_18_Geometry.crs is None:
    Total_18_Geometry = Total_18_Geometry.set_crs(Departamentos.crs, allow_override=True)

Total_18_Geometry = gpd.sjoin_nearest(Total_18_Geometry,  Departamentos, how='left', distance_col='distancia').drop(['AREA' , 'PERIMETER' , 'HECTARES' , 'DPTO','index_right' , 'distancia'] , axis=1)
Total_18_Geometry
Total_18_Geometry[Total_18_Geometry.isnull().any(axis=1)]

#Agregar Clima
clima_2018 = clima[clima['FechaObservacion'].dt.year == 2019]
clima_2018 = arreglar_texto(clima_2018 , 'Municipio', 'Ciudad')
clima_2018= completar_meses(clima_2018 , 'Ciudad' , 'Mes' ,'ValorObservado' , 0)
Total_18_Clima= pd.merge(Total_18_Geometry, clima_2018[['Ciudad', 'ValorObservado' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18_Clima = Total_18_Clima.rename(columns={'ValorObservado': 'Temperatura'})

#Krigging para el clima 
df_kriging = meses_a_numeros(Total_18_Clima , 'Mes')
df_kriging.set_crs(epsg=4326, inplace=True)
df_kriging=df_kriging.to_crs(epsg=32618)  

meses_krig = df_kriging['Mes'].unique()

for i in meses_krig:
    print('Procesando Mes:' ,i)
    
    df_kriging_mes = df_kriging[df_kriging['Mes'] == i] # Vamos a realizar el kriging por mes
    df_kriging_known = df_kriging_mes.dropna(subset=['Temperatura']) # Los datos para los que tenemos temperatura
    df_kriging_missing = df_kriging_mes[df_kriging_mes ['Temperatura'].isna()] # Los datos para los NO que tenemos temperatura (Variable a predecir)
    
    if df_kriging_missing.empty:
        print('No hay datos faltantes en el mes ' , i) # Si algun mes esta completo para todas las ciudades, no hacer nada 
        continue
    
    # Nuestras variables para predecir
    x_known = df_kriging_known.geometry.x.values
    y_known = df_kriging_known.geometry.y.values
    z_known = df_kriging_known['Temperatura'].values
    
    # Nuestros datos a predecir
    x_missing = df_kriging_missing.geometry.x.values
    y_missing = df_kriging_missing.geometry.y.values
    
    # Crear el modelo 
    Krigg = OrdinaryKriging(
        x_known, y_known, z_known,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False)
    
    # Realizar predicciones con el krigging
    z_pred, ss = Krigg.execute('points', x_missing, y_missing)
    
    # Introducir en el DF los valores
    df_kriging.loc[df_kriging_missing.index, 'Temperatura'] = z_pred

#Agregar precio dolar
Dolar = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Dolar.xlsx')
Dolar['Fecha'] = pd.to_datetime(Dolar['Fecha'], errors='coerce')
Dolar.set_index('Fecha', inplace=True)
Dolar = Dolar.resample('M').median()
Dolar = Dolar.reset_index()
Dolar_2018=Dolar[Dolar['Fecha'].dt.year == 2019]
Dolar_2018['Mes'] = range(1, 13)
Base_2018=pd.merge(df_kriging, Dolar_2018, on= 'Mes', how='right').drop('Fecha' , axis=1)
Base_2018

#Interpolar PIB mensual

PIB= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2019\\PIB - Miles de millones de pesos  - 2019.xlsx')
PIB= arreglar_texto(PIB, 'DEPARTAMENTO' , 'Departamento')
PIB['VALOR (unidades)'] = PIB['VALOR (unidades)'].str.replace(',', '.')
PIB['VALOR (unidades)'] = pd.to_numeric(PIB['VALOR (unidades)'], errors='coerce')

# Usaremos el metodo de LOESS para descomponer el PIB mensualmente 
meses_loees= np.arange(1, 13)
tendencia_mensual_inflacion = [3.15 ,3.01, 3.21 , 3.25 , 3.31 , 3.43 , 3.79 , 3.75 , 3.82 , 3.86 , 3.84 , 3.80] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual_inflacion) for valor in tendencia_mensual_inflacion]) 
patron_estacional = patron_estacional / patron_estacional.sum()

# La idea es crear algun tipo de ruido sobre nuestro patron, para asi tratar de simular efectos reales economicos.
np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) #  Asegurarnos que todos las fulctuaciones sean positivas. 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()


def descomponer_pib_loess(pib_anual, patron):
    def asignar_pib_mensual(k, patron):
        pib_mensual = patron * k['VALOR (unidades)']
        return pib_mensual

    df_mensual = pd.DataFrame()
    
    for i, k in pib_anual.iterrows():
        pib_mensual = asignar_pib_mensual(k, patron)
        
        df_temp = pd.DataFrame({
            'Departamento': k['Departamento'],
            'Mes': np.arange(1, 13),
            'PIB_Mensual': pib_mensual
        })
        
        df_mensual = pd.concat([df_mensual, df_temp], ignore_index=True)
    
    return df_mensual

pib_mensual = descomponer_pib_loess(PIB, patron_smooth)
importancia_pesos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\importancia-municipal.xlsx')

#Dado que encontrar una serie completa desde 2018 hasta 2024 de el valor del PIB por ciudad no es facil, primero usaremos la descomposicon por mes que hicimos departamental y la ponderaremos 
#por la importancia de cada ciudad en el departamento para encontrar un aproximando significativo. pdta= la base que nos pasaste profe no tenia todos los anos por eso hacemos esta aproximacion
importancia_pesos = arreglar_texto(importancia_pesos , 'Municipio / Distrito' , 'Ciudad')
importancia_pesos['Departamento'] = importancia_pesos['Departamento'].astype(str)
importancia_pesos = arreglar_texto(importancia_pesos , 'Departamento' , 'Departamento')

# Dejar solo las ciudades que estamos trabajando.
for i in importancia_pesos['Ciudad']:
    indice = importancia_pesos[importancia_pesos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        importancia_pesos = importancia_pesos.drop(index=indice)
    else:
        pass

importancia_pesos_mensual = importancia_pesos.loc[importancia_pesos.index.repeat(12)].reset_index(drop=True)
importancia_pesos_mensual['Mes'] = (importancia_pesos_mensual.index % 12) + 1
pib_pesos_ciudades1 = pd.merge(importancia_pesos_mensual, pib_mensual, on=['Departamento', 'Mes'], how='left')
pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] /100
pib_pesos_ciudades1['Pib Ponderado'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] * pib_pesos_ciudades1['PIB_Mensual']
fusion_left = pd.merge(Base_2018, pib_pesos_ciudades1[['Pib Ponderado' , 'Ciudad', 'Mes']], on=['Ciudad', 'Mes'], how='left')

#Agregar puntos de llegadas internacionales
entradas = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Entradas_de_extranjeros_a_Colombia_20241006.csv')
entradas_2018= entradas[entradas['Año']==2019]
entradas_2018 = entradas_2018[entradas_2018['Latitud - Longitud'] != 'No Aplica,No Aplica']
entradas_2018 =traducir(entradas_2018 , 'Mes')
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )
entradas_2018 =completar_meses(entradas_2018, 'Latitud - Longitud' , 'Mes' , 'Total' ,1 )
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )

entradas_geometry = convertir_ubicacion(entradas_2018 , 'Latitud - Longitud')
Entradas = gpd.GeoDataFrame(entradas_geometry, geometry='geometry')

if Entradas.crs is None:
    Entradas = Entradas.set_crs(Departamentos.crs, allow_override=True)

Entradas = gpd.sjoin_nearest(Entradas,  Departamentos, how='left', distance_col='distancia').drop(['index_right' , 'DPTO' , 'AREA' , 'PERIMETER' , 'HECTARES' ,'distancia'] , axis=1)
Entradas_Departamentos = Entradas.groupby(['Mes', 'Departamento'])['Total'].sum().reset_index()
final = pd.merge(fusion_left, Entradas_Departamentos, on=['Departamento', 'Mes'], how='left').rename(columns={'Total': 'Entradas Extranjeros Zona'})
final['Entradas Extranjeros Zona'] = final['Entradas Extranjeros Zona'].fillna(0)
final[final['Ciudad'] == 'medellin']

#Distancias desde los puntos de llegadas internacionales ponderadas 
for i in ciudades['Ciudad']:
    indice = ciudades[ciudades['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_unico = ciudades.drop(index=indice)
    else:
        pass

ciudades_unico = ciudades_unico.iloc[:, [0, -1]]
ciudades_unico= gpd.GeoDataFrame(ciudades_unico, geometry='geometry')
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')

if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if Migracion_Unico.crs is None:
    Migracion_Unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de Migracion_Unico: {Migracion_Unico.crs}")

ciudades_unico = ciudades_unico.to_crs(epsg=32618)
Migracion_Unico = Migracion_Unico.to_crs(epsg=32618)

ciudades_unico['key'] = 1
Migracion_Unico['key'] = 1

Distancia_combinado = pd.merge(ciudades_unico, Migracion_Unico, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

Distancia_combinado['distancia_km'] = Distancia_combinado.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_combinado['distancia_ponderada'] = Distancia_combinado['distancia_km'] * Distancia_combinado['Total']
Distancia_ponderado = Distancia_combinado.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_migrantes=('Total', 'sum')
).reset_index()

Distancia_ponderado['distancia_ponderada_km'] = Distancia_ponderado['suma_ponderada'] / Distancia_ponderado['suma_migrantes']
Distancia_ponderado_final = Distancia_ponderado[['Ciudad' , 'distancia_ponderada_km']]

Distancia_ciudades = ciudades_unico.merge(Distancia_ponderado_final, on='Ciudad')
final1 = pd.merge(final, Distancia_ciudades, on='Ciudad', how='left').drop(['geometry_y' , 'key'] , axis=1)

#Score de la importancia de los aeropuertos
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')
columnas = ['geometry' , 'Departamento' ,'Total' ]
Migracion_Unico_Dept = gpd.sjoin_nearest(Migracion_Unico, Departamentos, how='left', distance_col='distancia')[columnas]
Total_Migracion = Migracion_Unico_Dept.groupby('Departamento').agg(
    puntos=('Departamento', 'size'),  
    total_personas=('Total', 'sum')  
).reset_index()
scaler = MinMaxScaler()
Total_Migracion[['puntos_norm', 'total_norm']] = scaler.fit_transform(Total_Migracion[['puntos', 'total_personas']])
Total_Migracion['importancia accesos'] = 0.3 * Total_Migracion['puntos_norm'] + 0.7 * Total_Migracion['total_norm'] # Asumimos que es mas importante cuanta gente llega a que tantos puntos hay.
final1 = pd.merge(final1, Total_Migracion, on='Departamento', how='left').drop(['puntos' , 'total_personas' , 'puntos_norm' , 'total_norm'] , axis=1)
final1['importancia accesos'] = final1['importancia accesos'].fillna(0)

#Numero establecimientos de turismo, camas y habitaciones para turistas.
hoteles = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Hist_rico_Registro_Nacional_de_Turismo_-_RNT_20241007.csv')
establecimientos_2018 = hoteles[hoteles['AÑO']==2019]
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('PUERTO INIRIDA', 'Inirida', regex=False)
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('BUGA', 'guadalajaradebuga', regex=False)

Establecimientos = establecimientos_2018.groupby('NOMBRE-MUNI').agg({'CATEGORIA': 'count','HABITACIONES': 'sum', 'CAMAS': 'sum'}).reset_index().rename(columns={'CATEGORIA': 'Establecimientos'})
Establecimientos = arreglar_texto(Establecimientos, 'NOMBRE-MUNI' , 'Municipios')
for i in Establecimientos['Municipios']:
    indice = Establecimientos[Establecimientos['Municipios'] == i].index
    if i not in grandes_ciudades:
        Establecimientos = Establecimientos.drop(index=indice)
    else:
        pass

Establecimientos=Establecimientos.rename(columns={'Municipios': 'Ciudad'})
meses_loees = np.arange(1, 13)
tendencia_mensual = [30,33,36,35,36,38,39,40,41,42,43,43] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual) for valor in tendencia_mensual]) 
patron_estacional = patron_estacional / patron_estacional.sum()

np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()

def descomponer_variables_loess(df_anual, patron, variables):
    def asignar_valores_mensuales(fila, patron, variables):
        valores_mensuales = {}
        for i in variables:
            valores_mensuales[i] = patron * fila[i]
        return valores_mensuales

    data_mensual = []
    for i, fila in df_anual.iterrows():
        valores_mensuales = asignar_valores_mensuales(fila, patron, variables)
        
        df_temp = pd.DataFrame({
            'Ciudad': fila['Ciudad'],
            'Mes': np.arange(1, 13)
        })
        
        for k in variables:
            df_temp[k] = valores_mensuales[k]
        
        data_mensual.append(df_temp)
    
    df_mensual = pd.concat(data_mensual, ignore_index=True)
    
    return df_mensual

Establecimientos_mensual = descomponer_variables_loess(Establecimientos, patron_smooth , ['Establecimientos' , 'HABITACIONES' , 'CAMAS'])
final2 = pd.merge(final1, Establecimientos_mensual, on=['Ciudad', 'Mes'], how='left')

#Distancias al TOP de colombia 

df_TopColombia = pd.DataFrame.from_dict(Top_Colombia, orient='index', columns=['coordinates','Reviews', 'rating', 'eliminar'])
df_TopColombia = df_TopColombia.reset_index().rename(columns={'index': 'lugar'}).drop('eliminar' , axis=1)
df_TopColombia['geometry'] = df_TopColombia['coordinates'].apply(lambda x: Point(x[1], x[0]))
df_TopColombia_gdp = gpd.GeoDataFrame(df_TopColombia, geometry='geometry').drop('coordinates' , axis=1)
if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if df_TopColombia_gdp.crs is None:
    df_TopColombia_gdp.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de df_TopColombia_gdp: {df_TopColombia_gdp.crs}")


ciudades_unico = ciudades_unico.to_crs(epsg=32618)
df_TopColombia_gdp = df_TopColombia_gdp.to_crs(epsg=32618)

scaler = MinMaxScaler()

ciudades_unico['key'] = 1
df_TopColombia_gdp['key'] = 1

Distancia_Top = pd.merge(ciudades_unico, df_TopColombia_gdp, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

w_reviews = 0.6 # Vamos a asumir que las reviews tienen mas peso que la calificacion del lugar 
w_rating = 0.4

Distancia_Top['distancia_km'] = Distancia_Top.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_Top[['distancia_km','Reviews', 'rating']] = scaler.fit_transform(
    Distancia_Top[['distancia_km','Reviews', 'rating']]
) # Normalizamos las variables para calcular una distancia ponderada con las mismas escalas

Distancia_Top['distancia_ponderada'] = (Distancia_Top['distancia_km']) + ( w_reviews* Distancia_Top['Reviews']) + ( w_rating* Distancia_Top['rating'])

Distancia_Ponderado_Top = Distancia_Top.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_reviews=('Reviews', 'sum'),
    suma_rating=('rating', 'sum')).reset_index()

Distancia_Ponderado_Top['distancia_ponderada_TOP'] = Distancia_Ponderado_Top['suma_ponderada'] / (Distancia_Ponderado_Top['suma_reviews']+Distancia_Ponderado_Top['suma_rating'])
Distancia_Ponderado_Top_final = Distancia_Ponderado_Top[['Ciudad' , 'distancia_ponderada_TOP']]

Distancia_Top = ciudades_unico.merge(Distancia_Ponderado_Top_final, on='Ciudad')
final3 = pd.merge(final2, Distancia_Top, on='Ciudad', how='left').drop(['geometry_x' , 'key'] , axis=1)

# Pobreza con proxy
ciudades_poblacion = ciudades[['Ciudad' , 'population']]
for i in ciudades_poblacion['Ciudad']:
    indice = ciudades_poblacion[ciudades_poblacion['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_poblacion = ciudades_poblacion.drop(index=indice)
    else:
        pass

final4 = pd.merge(final3, ciudades_poblacion, on='Ciudad', how='left')
final4['Proxy_Pobreza'] = final4['Pib Ponderado'] / final4['population']
final4

#Gasto promedio por dia de un turista
gastos_diarios = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Gasto Promedio Diario 2019.xlsx')
gastos_diarios = arreglar_texto(gastos_diarios , 'Ciudad', 'Ciudad')
ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_diarios = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='left')

gastos_diarios_Nan = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_diarios_Nan = gpd.GeoDataFrame(pd.merge(gastos_diarios_Nan, ciudades_geometry, on='Ciudad', how='left').drop('geometry_x' , axis=1) , geometry='geometry')
gastos_diarios_Nan.set_crs(epsg=4326, inplace=True)
gastos_diarios_Nan=gastos_diarios_Nan.to_crs(epsg=32618)  
variables = [i for i in gastos_diarios_Nan.columns][1:-1]
gastos_sin_Nan = gastos_diarios_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_diarios_Nan[gastos_diarios_Nan[variables].isnull().any(axis=1)]

def kriging(df_con_info, variable, grid_size=100):
    coords = np.array(list(zip(df_con_info.geometry.x, df_con_info.geometry.y)))
    valores = df_con_info[variable].values
    
    min_x, min_y, max_x, max_y = gastos_diarios_Nan.total_bounds
    gridx = np.linspace(min_x, max_x, grid_size)
    gridy = np.linspace(min_y, max_y, grid_size)
    
    Krigg_O = OrdinaryKriging(
        coords[:,0], coords[:,1], valores,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False
    )
    
    z1, ss1 = Krigg_O.execute('grid', gridx, gridy)
    
    return Krigg_O, gridx, gridy, z1

def estimar(Krigg, puntos):
    x = puntos.geometry.x.values
    y = puntos.geometry.y.values
    estimados, ss = Krigg.execute('points', x, y)
    return estimados

for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual['Mes'] = (gastos_krigg_mensual.index % 12) + 1
final5 = pd.merge(final4, gastos_krigg_mensual, on=['Mes', 'Ciudad'], how='left').drop(['geometry_x','population'] , axis=1)

# Gastos de todo el viaje 
gastos_viaje = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Gasto Promedio Viaje 2019.xlsx')
gastos_viaje = arreglar_texto(gastos_viaje , 'Ciudad', 'Ciudad')

ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_viaje = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='left')
gastos_viaje_Nan = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_viaje_Nan = gpd.GeoDataFrame(pd.merge(gastos_viaje_Nan, ciudades_geometry, on='Ciudad', how='left') , geometry='geometry')
gastos_viaje_Nan = gastos_viaje_Nan.drop('geometry_x' , axis=1)

gastos_viaje_Nan.set_crs(epsg=4326, inplace=True)
gastos_viaje_Nan=gastos_viaje_Nan.to_crs(epsg=32618) 
variables = [i for i in gastos_viaje_Nan.columns][1:-1]
gastos_sin_Nan = gastos_viaje_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_viaje_Nan[gastos_viaje_Nan[variables].isnull().any(axis=1)]
for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual_viaje = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual_viaje['Mes'] = (gastos_krigg_mensual_viaje.index % 12) + 1
gastos_krigg_mensual_viaje
Base_2019 = pd.merge(final5, gastos_krigg_mensual_viaje, on=['Mes', 'Ciudad'], how='left').drop('geometry_y' , axis=1)

# Agregar inflacion
inflacion = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Inflacion.xlsx')
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion[inflacion['Fecha'].dt.year == 2019].rename(columns={'Fecha':'Mes'})
inflacion['Mes'] = inflacion['Mes'].dt.month
Base_2019 = pd.merge(Base_2019, inflacion, on='Mes', how='left')

#Agregar Vias
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2019 = pd.merge(Base_2019 , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

# Agregar Eventos
eventos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Eventos Turisticos\\Eventos 2018.xlsx').fillna(0)
eventos = arreglar_texto(eventos ,'Departamento' , 'Departamento')
eventos['Departamento'] = eventos['Departamento'].str.replace('sanandres', 'archipielagodesanandresprovidenciaysantacatalina', regex=False)
Base_2019 = pd.merge(Base_2019 , eventos , on=['Departamento' ,'Mes'] , how='left')

## Lets see 2019 DataFrame 

In [14]:
Base_2019

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos
0,girardot,1,27.0,3,58,8,cundinamarca,28.700000,3150.58,121.451194,...,25988.446737,357933.270016,56527.540955,81846.745115,91748.852583,140826.671904,POINT (522004.027 475843.689),3.1500,9,1.0
1,popayan,1,320.0,7,257,31,cauca,16.300000,3150.58,357.165903,...,31928.992000,439015.379000,46166.826000,92300.690000,140487.972000,160059.891000,POINT (321075.878 271372.118),3.1500,8,0.0
2,maicao,1,40.0,7,35,3,laguajira,21.829001,3150.58,88.008349,...,26593.785064,385184.040015,51688.611118,94016.628038,91748.852583,167410.817311,POINT (801350.843 1259186.753),3.1500,2,0.0
3,fusagasuga,1,58.0,1,31,12,cundinamarca,21.829001,3150.58,148.135521,...,27638.508499,386477.528213,64397.503747,84766.242979,91748.852583,171105.701543,POINT (570814.377 480314.274),3.1500,3,1.0
4,barranquilla,1,4487.0,25,879,72,atlantico,25.600000,3150.58,2365.517791,...,32911.341000,416462.359000,85596.957000,77403.016000,107659.272000,145803.114000,POINT (521642.463 1214140.036),3.1500,10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,turbaco,12,33.0,5,40,6,bolivar,21.529886,3364.24,68.413769,...,28376.075240,385025.774591,58880.717603,78301.672584,91748.852583,152516.873607,POINT (463511.39 1144129.202),3.8043,4,0.0
968,magangue,12,15.0,2,18,6,bolivar,21.529886,3364.24,132.044220,...,27183.813357,371202.047827,60773.505977,83144.027852,91748.852583,128300.836444,POINT (525625.153 1022500.055),3.8043,2,0.0
969,uribia,12,0.0,0,4,0,laguajira,29.900000,3364.24,62.083220,...,26664.730192,386623.688304,49480.398965,94124.673761,91748.852583,172041.263678,POINT (826814.742 1319110.428),3.8043,0,1.0
970,palermo,12,1.0,2,5,3,huila,21.529886,3364.24,57.558129,...,23492.093992,319443.333857,40252.245211,69580.193582,91748.852583,91297.020176,POINT (451378.305 319631.823),3.8043,2,0.0


In [15]:
Base_2019[Base_2019.isna().any(axis=1)]

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos


# 2021

In [ ]:
#Inicio proceso
df= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Visitantes_No_Residentes.csv')
df1 = df[df["Año"] == 2021]
df1= arreglar_texto(df1 , 'Ciudad' , 'Ciudad')
Homicidios= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\homicidios_9.xls').dropna()
Homicidos1 = bases_crimenes(Homicidios , 'FECHA ' , 'CANTIDAD ' , 'Homicidios')
Hurto= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\hurto_a_personas_9.xlsx').dropna()
Hurto1 = bases_crimenes(Hurto , 'FECHA  HECHO ' , 'CANTIDAD ' , 'Hurtos')
sexuales= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\delitos_sexuales_9.xls').dropna()
sexuales1 = bases_crimenes(sexuales , 'FECHA ' , 'CANTIDAD ' , 'Delitos Sexuales')
ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades = arreglar_texto(ciudades, 'city' , "Ciudad")
ciudades['admin_name'] = ciudades['admin_name'].astype(str)
ciudades.at[68, 'admin_name'] = 'Atlantico'
Total_18= completar_meses(df1 , 'Ciudad' , 'Mes' ,'Extranjeros no Residentes' , 1)
Total_18 = pd.merge(Total_18, Homicidos1[['Ciudad', 'Homicidios' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, Hurto1[['Ciudad', 'Hurtos' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, sexuales1[['Ciudad', 'Delitos Sexuales' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18['Homicidios'] = Total_18['Homicidios'].fillna(0)
Total_18['Hurtos'] = Total_18['Hurtos'].fillna(0)
Total_18['Delitos Sexuales'] = Total_18['Delitos Sexuales'].fillna(0)
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('bogotadc', 'bogota', regex=False)

#Seleccionar Ciudades que vamos a trabajar
ciudades1= set(ciudades['Ciudad'])
ciudades2= set(Total_18['Ciudad'])
grandes_ciudades=list(ciudades1.intersection(ciudades2))
l=['bogota' , 'tumaco']
grandes_ciudades = grandes_ciudades + l
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('sanandresdetumaco', 'tumaco', regex=False)
for i in Total_18['Ciudad']:
    indice = Total_18[Total_18['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Total_18 = Total_18.drop(index=indice)
    else:
        pass

#Poner Ubicaciones
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades_geometry= gpd.GeoDataFrame(ciudades, geometry='geometry')
Total_18_Geometry= pd.merge(Total_18, ciudades_geometry[['Ciudad', 'geometry']], on='Ciudad' , how='left')
Total_18_Geometry= gpd.GeoDataFrame(Total_18_Geometry, geometry='geometry')

# Poner departamentos
Departamentos = gpd.read_file("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Colombia.json")
Departamentos = arreglar_texto(Departamentos, 'NOMBRE_DPT' , "Departamento")
Departamentos["Departamento"] = Departamentos["Departamento"].str.replace('santafedebogota', 'bogotadc', regex=False)

if Total_18_Geometry.crs is None:
    Total_18_Geometry = Total_18_Geometry.set_crs(Departamentos.crs, allow_override=True)

Total_18_Geometry = gpd.sjoin_nearest(Total_18_Geometry,  Departamentos, how='left', distance_col='distancia').drop(['AREA' , 'PERIMETER' , 'HECTARES' , 'DPTO','index_right' , 'distancia'] , axis=1)
Total_18_Geometry
Total_18_Geometry[Total_18_Geometry.isnull().any(axis=1)]

#Agregar Clima
clima_2018 = clima[clima['FechaObservacion'].dt.year == 2021]
clima_2018 = arreglar_texto(clima_2018 , 'Municipio', 'Ciudad')
clima_2018= completar_meses(clima_2018 , 'Ciudad' , 'Mes' ,'ValorObservado' , 0)
Total_18_Clima= pd.merge(Total_18_Geometry, clima_2018[['Ciudad', 'ValorObservado' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18_Clima = Total_18_Clima.rename(columns={'ValorObservado': 'Temperatura'})

#Krigging para el clima 
df_kriging = meses_a_numeros(Total_18_Clima , 'Mes')
df_kriging.set_crs(epsg=4326, inplace=True)
df_kriging=df_kriging.to_crs(epsg=32618)  

meses_krig = df_kriging['Mes'].unique()

for i in meses_krig:
    print('Procesando Mes:' ,i)
    
    df_kriging_mes = df_kriging[df_kriging['Mes'] == i] # Vamos a realizar el kriging por mes
    df_kriging_known = df_kriging_mes.dropna(subset=['Temperatura']) # Los datos para los que tenemos temperatura
    df_kriging_missing = df_kriging_mes[df_kriging_mes ['Temperatura'].isna()] # Los datos para los NO que tenemos temperatura (Variable a predecir)
    
    if df_kriging_missing.empty:
        print('No hay datos faltantes en el mes ' , i) # Si algun mes esta completo para todas las ciudades, no hacer nada 
        continue
    
    # Nuestras variables para predecir
    x_known = df_kriging_known.geometry.x.values
    y_known = df_kriging_known.geometry.y.values
    z_known = df_kriging_known['Temperatura'].values
    
    # Nuestros datos a predecir
    x_missing = df_kriging_missing.geometry.x.values
    y_missing = df_kriging_missing.geometry.y.values
    
    # Crear el modelo 
    Krigg = OrdinaryKriging(
        x_known, y_known, z_known,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False)
    
    # Realizar predicciones con el krigging
    z_pred, ss = Krigg.execute('points', x_missing, y_missing)
    
    # Introducir en el DF los valores
    df_kriging.loc[df_kriging_missing.index, 'Temperatura'] = z_pred

#Agregar precio dolar
Dolar = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Dolar.xlsx')
Dolar['Fecha'] = pd.to_datetime(Dolar['Fecha'], errors='coerce')
Dolar.set_index('Fecha', inplace=True)
Dolar = Dolar.resample('M').median()
Dolar = Dolar.reset_index()
Dolar_2018=Dolar[Dolar['Fecha'].dt.year == 2021]
Dolar_2018['Mes'] = range(1, 13)
Base_2018=pd.merge(df_kriging, Dolar_2018, on= 'Mes', how='right').drop('Fecha' , axis=1)
Base_2018

#Interpolar PIB mensual

PIB = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\PIB - Miles de millones de pesos  - 2021p.xlsx')
PIB= arreglar_texto(PIB, 'DEPARTAMENTO' , 'Departamento')
PIB['VALOR (unidades)'] = PIB['VALOR (unidades)'].str.replace(',', '.')
PIB['VALOR (unidades)'] = pd.to_numeric(PIB['VALOR (unidades)'], errors='coerce')
# Usaremos el metodo de LOESS para descomponer el PIB mensualmente 
meses_loees= np.arange(1, 13)
tendencia_mensual_inflacion = [1.6 ,1.56, 1.51 , 1.95 , 3.3 , 3.63 , 3.97 , 4.44 , 4.51 , 4.58 , 5.26 , 5.62] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual_inflacion) for valor in tendencia_mensual_inflacion]) 
patron_estacional = patron_estacional / patron_estacional.sum()

# La idea es crear algun tipo de ruido sobre nuestro patron, para asi tratar de simular efectos reales economicos.
np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) #  Asegurarnos que todos las fulctuaciones sean positivas. 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()


def descomponer_pib_loess(pib_anual, patron):
    def asignar_pib_mensual(k, patron):
        pib_mensual = patron * k['VALOR (unidades)']
        return pib_mensual

    df_mensual = pd.DataFrame()
    
    for i, k in pib_anual.iterrows():
        pib_mensual = asignar_pib_mensual(k, patron)
        
        df_temp = pd.DataFrame({
            'Departamento': k['Departamento'],
            'Mes': np.arange(1, 13),
            'PIB_Mensual': pib_mensual
        })
        
        df_mensual = pd.concat([df_mensual, df_temp], ignore_index=True)
    
    return df_mensual

pib_mensual = descomponer_pib_loess(PIB, patron_smooth)
importancia_pesos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\importancia-municipal.xlsx')

#Dado que encontrar una serie completa desde 2018 hasta 2024 de el valor del PIB por ciudad no es facil, primero usaremos la descomposicon por mes que hicimos departamental y la ponderaremos 
#por la importancia de cada ciudad en el departamento para encontrar un aproximando significativo. pdta= la base que nos pasaste profe no tenia todos los anos por eso hacemos esta aproximacion
importancia_pesos = arreglar_texto(importancia_pesos , 'Municipio / Distrito' , 'Ciudad')
importancia_pesos['Departamento'] = importancia_pesos['Departamento'].astype(str)
importancia_pesos = arreglar_texto(importancia_pesos , 'Departamento' , 'Departamento')

# Dejar solo las ciudades que estamos trabajando.
for i in importancia_pesos['Ciudad']:
    indice = importancia_pesos[importancia_pesos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        importancia_pesos = importancia_pesos.drop(index=indice)
    else:
        pass

importancia_pesos_mensual = importancia_pesos.loc[importancia_pesos.index.repeat(12)].reset_index(drop=True)
importancia_pesos_mensual['Mes'] = (importancia_pesos_mensual.index % 12) + 1
pib_pesos_ciudades1 = pd.merge(importancia_pesos_mensual, pib_mensual, on=['Departamento', 'Mes'], how='left')
pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] /100
pib_pesos_ciudades1['Pib Ponderado'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] * pib_pesos_ciudades1['PIB_Mensual']
fusion_left = pd.merge(Base_2018, pib_pesos_ciudades1[['Pib Ponderado' , 'Ciudad', 'Mes']], on=['Ciudad', 'Mes'], how='left')

#Agregar puntos de llegadas internacionales
entradas = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Entradas_de_extranjeros_a_Colombia_20241006.csv')
entradas_2018= entradas[entradas['Año']==2021]
entradas_2018 = entradas_2018[entradas_2018['Latitud - Longitud'] != 'No Aplica,No Aplica']
entradas_2018 =traducir(entradas_2018 , 'Mes')
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )
entradas_2018 =completar_meses(entradas_2018, 'Latitud - Longitud' , 'Mes' , 'Total' ,1 )
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )

entradas_geometry = convertir_ubicacion(entradas_2018 , 'Latitud - Longitud')
Entradas = gpd.GeoDataFrame(entradas_geometry, geometry='geometry')

if Entradas.crs is None:
    Entradas = Entradas.set_crs(Departamentos.crs, allow_override=True)

Entradas = gpd.sjoin_nearest(Entradas,  Departamentos, how='left', distance_col='distancia').drop(['index_right' , 'DPTO' , 'AREA' , 'PERIMETER' , 'HECTARES' ,'distancia'] , axis=1)
Entradas_Departamentos = Entradas.groupby(['Mes', 'Departamento'])['Total'].sum().reset_index()
final = pd.merge(fusion_left, Entradas_Departamentos, on=['Departamento', 'Mes'], how='left').rename(columns={'Total': 'Entradas Extranjeros Zona'})
final['Entradas Extranjeros Zona'] = final['Entradas Extranjeros Zona'].fillna(0)
final[final['Ciudad'] == 'medellin']

#Distancias desde los puntos de llegadas internacionales ponderadas 
for i in ciudades['Ciudad']:
    indice = ciudades[ciudades['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_unico = ciudades.drop(index=indice)
    else:
        pass

ciudades_unico = ciudades_unico.iloc[:, [0, -1]]
ciudades_unico= gpd.GeoDataFrame(ciudades_unico, geometry='geometry')
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')

if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if Migracion_Unico.crs is None:
    Migracion_Unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de Migracion_Unico: {Migracion_Unico.crs}")

ciudades_unico = ciudades_unico.to_crs(epsg=32618)
Migracion_Unico = Migracion_Unico.to_crs(epsg=32618)

ciudades_unico['key'] = 1
Migracion_Unico['key'] = 1

Distancia_combinado = pd.merge(ciudades_unico, Migracion_Unico, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

Distancia_combinado['distancia_km'] = Distancia_combinado.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_combinado['distancia_ponderada'] = Distancia_combinado['distancia_km'] * Distancia_combinado['Total']
Distancia_ponderado = Distancia_combinado.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_migrantes=('Total', 'sum')
).reset_index()

Distancia_ponderado['distancia_ponderada_km'] = Distancia_ponderado['suma_ponderada'] / Distancia_ponderado['suma_migrantes']
Distancia_ponderado_final = Distancia_ponderado[['Ciudad' , 'distancia_ponderada_km']]

Distancia_ciudades = ciudades_unico.merge(Distancia_ponderado_final, on='Ciudad')
final1 = pd.merge(final, Distancia_ciudades, on='Ciudad', how='left').drop(['geometry_y' , 'key'] , axis=1)

# Score importancia puntos entrada
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')
columnas = ['geometry' , 'Departamento' ,'Total' ]
Migracion_Unico_Dept = gpd.sjoin_nearest(Migracion_Unico, Departamentos, how='left', distance_col='distancia')[columnas]
Total_Migracion = Migracion_Unico_Dept.groupby('Departamento').agg(
    puntos=('Departamento', 'size'),  
    total_personas=('Total', 'sum')  
).reset_index()
scaler = MinMaxScaler()
Total_Migracion[['puntos_norm', 'total_norm']] = scaler.fit_transform(Total_Migracion[['puntos', 'total_personas']])
Total_Migracion['importancia accesos'] = 0.3 * Total_Migracion['puntos_norm'] + 0.7 * Total_Migracion['total_norm'] # Asumimos que es mas importante cuanta gente llega a que tantos puntos hay.
final1 = pd.merge(final1, Total_Migracion, on='Departamento', how='left').drop(['puntos' , 'total_personas' , 'puntos_norm' , 'total_norm'] , axis=1)
final1['importancia accesos'] = final1['importancia accesos'].fillna(0)


#Numero establecimientos de turismo, camas y habitaciones para turistas.
hoteles = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Hist_rico_Registro_Nacional_de_Turismo_-_RNT_20241007.csv')
establecimientos_2018 = hoteles[hoteles['AÑO']==2021]
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('PUERTO INIRIDA', 'Inirida', regex=False)
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('BUGA', 'guadalajaradebuga', regex=False)

Establecimientos = establecimientos_2018.groupby('NOMBRE-MUNI').agg({'CATEGORIA': 'count','HABITACIONES': 'sum', 'CAMAS': 'sum'}).reset_index().rename(columns={'CATEGORIA': 'Establecimientos'})
Establecimientos = arreglar_texto(Establecimientos, 'NOMBRE-MUNI' , 'Municipios')
for i in Establecimientos['Municipios']:
    indice = Establecimientos[Establecimientos['Municipios'] == i].index
    if i not in grandes_ciudades:
        Establecimientos = Establecimientos.drop(index=indice)
    else:
        pass

Establecimientos=Establecimientos.rename(columns={'Municipios': 'Ciudad'})
meses_loees = np.arange(1, 13)
tendencia_mensual = [42,43,47,40,41,42,43,45,46,47,50,54] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual) for valor in tendencia_mensual]) 
patron_estacional = patron_estacional / patron_estacional.sum()

np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()

def descomponer_variables_loess(df_anual, patron, variables):
    def asignar_valores_mensuales(fila, patron, variables):
        valores_mensuales = {}
        for i in variables:
            valores_mensuales[i] = patron * fila[i]
        return valores_mensuales

    data_mensual = []
    for i, fila in df_anual.iterrows():
        valores_mensuales = asignar_valores_mensuales(fila, patron, variables)
        
        df_temp = pd.DataFrame({
            'Ciudad': fila['Ciudad'],
            'Mes': np.arange(1, 13)
        })
        
        for k in variables:
            df_temp[k] = valores_mensuales[k]
        
        data_mensual.append(df_temp)
    
    df_mensual = pd.concat(data_mensual, ignore_index=True)
    
    return df_mensual

Establecimientos_mensual = descomponer_variables_loess(Establecimientos, patron_smooth , ['Establecimientos' , 'HABITACIONES' , 'CAMAS'])
final2 = pd.merge(final1, Establecimientos_mensual, on=['Ciudad', 'Mes'], how='left')

#Distancias al TOP de colombia 
df_TopColombia = pd.DataFrame.from_dict(Top_Colombia, orient='index', columns=['coordinates','Reviews', 'rating', 'eliminar'])
df_TopColombia = df_TopColombia.reset_index().rename(columns={'index': 'lugar'}).drop('eliminar' , axis=1)
df_TopColombia['geometry'] = df_TopColombia['coordinates'].apply(lambda x: Point(x[1], x[0]))
df_TopColombia_gdp = gpd.GeoDataFrame(df_TopColombia, geometry='geometry').drop('coordinates' , axis=1)
if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if df_TopColombia_gdp.crs is None:
    df_TopColombia_gdp.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de df_TopColombia_gdp: {df_TopColombia_gdp.crs}")


ciudades_unico = ciudades_unico.to_crs(epsg=32618)
df_TopColombia_gdp = df_TopColombia_gdp.to_crs(epsg=32618)

scaler = MinMaxScaler()

ciudades_unico['key'] = 1
df_TopColombia_gdp['key'] = 1

Distancia_Top = pd.merge(ciudades_unico, df_TopColombia_gdp, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

w_reviews = 0.6 # Vamos a asumir que las reviews tienen mas peso que la calificacion del lugar 
w_rating = 0.4

Distancia_Top['distancia_km'] = Distancia_Top.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_Top[['distancia_km','Reviews', 'rating']] = scaler.fit_transform(
    Distancia_Top[['distancia_km','Reviews', 'rating']]
) # Normalizamos las variables para calcular una distancia ponderada con las mismas escalas

Distancia_Top['distancia_ponderada'] = (Distancia_Top['distancia_km']) + ( w_reviews* Distancia_Top['Reviews']) + ( w_rating* Distancia_Top['rating'])

Distancia_Ponderado_Top = Distancia_Top.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_reviews=('Reviews', 'sum'),
    suma_rating=('rating', 'sum')).reset_index()

Distancia_Ponderado_Top['distancia_ponderada_TOP'] = Distancia_Ponderado_Top['suma_ponderada'] / (Distancia_Ponderado_Top['suma_reviews']+Distancia_Ponderado_Top['suma_rating'])
Distancia_Ponderado_Top_final = Distancia_Ponderado_Top[['Ciudad' , 'distancia_ponderada_TOP']]

Distancia_Top = ciudades_unico.merge(Distancia_Ponderado_Top_final, on='Ciudad')
final3 = pd.merge(final2, Distancia_Top, on='Ciudad', how='left').drop(['geometry_x' , 'key'] , axis=1)

# Pobreza con proxy
ciudades_poblacion = ciudades[['Ciudad' , 'population']]
for i in ciudades_poblacion['Ciudad']:
    indice = ciudades_poblacion[ciudades_poblacion['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_poblacion = ciudades_poblacion.drop(index=indice)
    else:
        pass

final4 = pd.merge(final3, ciudades_poblacion, on='Ciudad', how='left')
final4['Proxy_Pobreza'] = final4['Pib Ponderado'] / final4['population']
final4

#Gasto promedio por dia de un turista
gastos_diarios = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\Gasto Promedio Diario 2021.xlsx')
gastos_diarios = arreglar_texto(gastos_diarios , 'Ciudad', 'Ciudad')
ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_diarios = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='left')

gastos_diarios_Nan = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_diarios_Nan = gpd.GeoDataFrame(pd.merge(gastos_diarios_Nan, ciudades_geometry, on='Ciudad', how='left').drop('geometry_x' , axis=1) , geometry='geometry')
gastos_diarios_Nan.set_crs(epsg=4326, inplace=True)
gastos_diarios_Nan=gastos_diarios_Nan.to_crs(epsg=32618)  
variables = [i for i in gastos_diarios_Nan.columns][1:-1]
gastos_sin_Nan = gastos_diarios_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_diarios_Nan[gastos_diarios_Nan[variables].isnull().any(axis=1)]

def kriging(df_con_info, variable, grid_size=100):
    coords = np.array(list(zip(df_con_info.geometry.x, df_con_info.geometry.y)))
    valores = df_con_info[variable].values
    
    min_x, min_y, max_x, max_y = gastos_diarios_Nan.total_bounds
    gridx = np.linspace(min_x, max_x, grid_size)
    gridy = np.linspace(min_y, max_y, grid_size)
    
    Krigg_O = OrdinaryKriging(
        coords[:,0], coords[:,1], valores,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False
    )
    
    z1, ss1 = Krigg_O.execute('grid', gridx, gridy)
    
    return Krigg_O, gridx, gridy, z1

def estimar(Krigg, puntos):
    x = puntos.geometry.x.values
    y = puntos.geometry.y.values
    estimados, ss = Krigg.execute('points', x, y)
    return estimados

for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual['Mes'] = (gastos_krigg_mensual.index % 12) + 1
final5 = pd.merge(final4, gastos_krigg_mensual, on=['Mes', 'Ciudad'], how='left').drop(['geometry_x','population'] , axis=1)

# Gastos de todo el viaje 
gastos_viaje = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2021\\Gasto Pormedio Viaje.xlsx')
gastos_viaje = arreglar_texto(gastos_viaje , 'Ciudad', 'Ciudad')

ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_viaje = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='left')
gastos_viaje_Nan = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_viaje_Nan = gpd.GeoDataFrame(pd.merge(gastos_viaje_Nan, ciudades_geometry, on='Ciudad', how='left') , geometry='geometry')
gastos_viaje_Nan = gastos_viaje_Nan.drop('geometry_x' , axis=1)

gastos_viaje_Nan.set_crs(epsg=4326, inplace=True)
gastos_viaje_Nan=gastos_viaje_Nan.to_crs(epsg=32618) 
variables = [i for i in gastos_viaje_Nan.columns][1:-1]
gastos_sin_Nan = gastos_viaje_Nan.dropna()
gastos_con_Nan = gastos_viaje_Nan[gastos_viaje_Nan[variables].isnull().any(axis=1)]
for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual_viaje = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual_viaje['Mes'] = (gastos_krigg_mensual_viaje.index % 12) + 1
gastos_krigg_mensual_viaje
Base_2021 = pd.merge(final5, gastos_krigg_mensual_viaje, on=['Mes', 'Ciudad'], how='left').drop('geometry_y' , axis=1)

# Agregar inflacion
inflacion = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Inflacion.xlsx')
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion[inflacion['Fecha'].dt.year == 2021].rename(columns={'Fecha':'Mes'})
inflacion['Mes'] = inflacion['Mes'].dt.month
Base_2021  = pd.merge(Base_2021 , inflacion, on='Mes', how='left')

#Agregar Vias
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2021 = pd.merge(Base_2021 , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

# Agregar Eventos
eventos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Eventos Turisticos\\Eventos 2021.xlsx').fillna(0)
eventos = arreglar_texto(eventos ,'Departamento' , 'Departamento')
eventos['Departamento'] = eventos['Departamento'].str.replace('sanandres', 'archipielagodesanandresprovidenciaysantacatalina', regex=False)
Base_2021 = pd.merge(Base_2021 , eventos , on=['Departamento' ,'Mes'] , how='left')

## Lets see 2021 DataFrame 

In [17]:
Base_2021

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos
0,cienaga,1,9.0,9,13,4,magdalena,22.672996,3478.11,55.531683,...,38309.988310,509576.010704,94227.742607,114452.575274,126513.416828,198360.186342,POINT (581933.159 1216844.759),1.59721,1,0.0
1,leticia,1,48.0,2,5,5,amazonas,24.700000,3478.11,23.000426,...,38349.247334,642297.592149,155336.389361,101229.841007,138644.470640,269015.023058,POINT (1063012.099 -467914.527),1.59721,1,0.0
2,bello,1,166.0,7,158,22,antioquia,21.064175,3478.11,258.040403,...,22500.262354,409266.901336,69171.990677,72707.583734,156302.284798,117579.038487,POINT (437321.773 700082.696),1.59721,3,0.0
3,barranquilla,1,2212.0,23,764,58,atlantico,25.800000,3478.11,1411.317456,...,46960.260000,797692.580000,161345.770000,143819.680000,243722.200000,248804.920000,POINT (521642.463 1214140.036),1.59721,10,0.0
4,cartagena,1,7563.0,20,439,50,bolivar,22.497558,3478.11,1160.904575,...,18030.146000,390658.100000,40006.560000,95969.190000,148800.860000,105881.480000,POINT (445269.937 1149681.547),1.59721,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,inirida,12,5.0,1,7,3,guainia,25.600000,3981.16,47.053040,...,37186.376345,667988.742264,163647.571594,106103.938719,154151.458844,256585.689974,POINT (1287613.509 430529.596),5.62267,0,0.0
968,puertocarreno,12,21.0,0,3,0,vichada,21.652402,3981.16,27.312103,...,36986.929789,674590.994187,164518.617859,105557.797928,157322.460242,258695.331397,POINT (1333897.888 690164.666),5.62267,1,1.0
969,mitu,12,1.0,0,6,3,vaupes,21.652402,3981.16,27.794043,...,31866.679879,580733.566611,133979.208194,109251.027751,135601.667940,215172.536627,POINT (1037614.182 132922.769),5.62267,0,0.0
970,palermo,12,10.0,0,4,0,huila,21.652402,3981.16,99.608632,...,23004.827084,424225.932035,87017.473048,90239.961293,106987.787639,120250.704847,POINT (451378.305 319631.823),5.62267,2,0.0


In [18]:
Base_2021[Base_2021.isna().any(axis=1)]


,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos


# 2022

In [ ]:
#Inicio proceso
df= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Visitantes_No_Residentes.csv')
df1 = df[df["Año"] == 2022]
df1= arreglar_texto(df1 , 'Ciudad' , 'Ciudad')
Homicidios= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\homicidios_7.xls').dropna()
Homicidos1 = bases_crimenes(Homicidios , 'FECHA HECHO' , 'CANTIDAD' , 'Homicidios')
Hurto= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\hurto_a_personas_17.xlsx').dropna()
Hurto1 = bases_crimenes(Hurto , 'FECHA HECHO' , 'CANTIDAD' , 'Hurtos')
sexuales= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\delitos_sexuales_11.xls').dropna()
sexuales1 = bases_crimenes(sexuales , 'FECHA HECHO' , 'CANTIDAD' , 'Delitos Sexuales')
ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades = arreglar_texto(ciudades, 'city' , "Ciudad")
ciudades['admin_name'] = ciudades['admin_name'].astype(str)
ciudades.at[68, 'admin_name'] = 'Atlantico'
Total_18= completar_meses(df1 , 'Ciudad' , 'Mes' ,'Extranjeros no Residentes' , 1)
Total_18 = pd.merge(Total_18, Homicidos1[['Ciudad', 'Homicidios' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, Hurto1[['Ciudad', 'Hurtos' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, sexuales1[['Ciudad', 'Delitos Sexuales' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18['Homicidios'] = Total_18['Homicidios'].fillna(0)
Total_18['Hurtos'] = Total_18['Hurtos'].fillna(0)
Total_18['Delitos Sexuales'] = Total_18['Delitos Sexuales'].fillna(0)
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('bogotadc', 'bogota', regex=False)

#Seleccionar Ciudades que vamos a trabajar
ciudades1= set(ciudades['Ciudad'])
ciudades2= set(Total_18['Ciudad'])
grandes_ciudades=list(ciudades1.intersection(ciudades2))
l=['bogota' , 'tumaco']
grandes_ciudades = grandes_ciudades + l
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('sanandresdetumaco', 'tumaco', regex=False)
for i in Total_18['Ciudad']:
    indice = Total_18[Total_18['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Total_18 = Total_18.drop(index=indice)
    else:
        pass

#Poner Ubicaciones
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades_geometry= gpd.GeoDataFrame(ciudades, geometry='geometry')
Total_18_Geometry= pd.merge(Total_18, ciudades_geometry[['Ciudad', 'geometry']], on='Ciudad' , how='left')
Total_18_Geometry= gpd.GeoDataFrame(Total_18_Geometry, geometry='geometry')

# Poner departamentos
Departamentos = gpd.read_file("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Colombia.json")
Departamentos = arreglar_texto(Departamentos, 'NOMBRE_DPT' , "Departamento")
Departamentos["Departamento"] = Departamentos["Departamento"].str.replace('santafedebogota', 'bogotadc', regex=False)

if Total_18_Geometry.crs is None:
    Total_18_Geometry = Total_18_Geometry.set_crs(Departamentos.crs, allow_override=True)

Total_18_Geometry = gpd.sjoin_nearest(Total_18_Geometry,  Departamentos, how='left', distance_col='distancia').drop(['AREA' , 'PERIMETER' , 'HECTARES' , 'DPTO','index_right' , 'distancia'] , axis=1)
Total_18_Geometry
Total_18_Geometry[Total_18_Geometry.isnull().any(axis=1)]

#Agregar Clima
clima_2018 = clima[clima['FechaObservacion'].dt.year == 2022]
clima_2018 = arreglar_texto(clima_2018 , 'Municipio', 'Ciudad')
clima_2018= completar_meses(clima_2018 , 'Ciudad' , 'Mes' ,'ValorObservado' , 0)
Total_18_Clima= pd.merge(Total_18_Geometry, clima_2018[['Ciudad', 'ValorObservado' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18_Clima = Total_18_Clima.rename(columns={'ValorObservado': 'Temperatura'})

#Krigging para el clima 
df_kriging = meses_a_numeros(Total_18_Clima , 'Mes')
df_kriging.set_crs(epsg=4326, inplace=True)
df_kriging=df_kriging.to_crs(epsg=32618)  

meses_krig = df_kriging['Mes'].unique()

for i in meses_krig:
    print('Procesando Mes:' ,i)
    
    df_kriging_mes = df_kriging[df_kriging['Mes'] == i] # Vamos a realizar el kriging por mes
    df_kriging_known = df_kriging_mes.dropna(subset=['Temperatura']) # Los datos para los que tenemos temperatura
    df_kriging_missing = df_kriging_mes[df_kriging_mes ['Temperatura'].isna()] # Los datos para los NO que tenemos temperatura (Variable a predecir)
    
    if df_kriging_missing.empty:
        print('No hay datos faltantes en el mes ' , i) # Si algun mes esta completo para todas las ciudades, no hacer nada 
        continue
    
    # Nuestras variables para predecir
    x_known = df_kriging_known.geometry.x.values
    y_known = df_kriging_known.geometry.y.values
    z_known = df_kriging_known['Temperatura'].values
    
    # Nuestros datos a predecir
    x_missing = df_kriging_missing.geometry.x.values
    y_missing = df_kriging_missing.geometry.y.values
    
    # Crear el modelo 
    Krigg = OrdinaryKriging(
        x_known, y_known, z_known,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False)
    
    # Realizar predicciones con el krigging
    z_pred, ss = Krigg.execute('points', x_missing, y_missing)
    
    # Introducir en el DF los valores
    df_kriging.loc[df_kriging_missing.index, 'Temperatura'] = z_pred

#Agregar precio dolar
Dolar = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Dolar.xlsx')
Dolar['Fecha'] = pd.to_datetime(Dolar['Fecha'], errors='coerce')
Dolar.set_index('Fecha', inplace=True)
Dolar = Dolar.resample('M').median()
Dolar = Dolar.reset_index()
Dolar_2018=Dolar[Dolar['Fecha'].dt.year == 2022]
Dolar_2018['Mes'] = range(1, 13)
Base_2018=pd.merge(df_kriging, Dolar_2018, on= 'Mes', how='right').drop('Fecha' , axis=1)
Base_2018

#Interpolar PIB mensual

PIB = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\PIB - Miles de millones de pesos  - 2022p.xlsx')
PIB= arreglar_texto(PIB, 'DEPARTAMENTO' , 'Departamento')
PIB['VALOR (unidades)'] = PIB['VALOR (unidades)'].str.replace(',', '.')
PIB['VALOR (unidades)'] = pd.to_numeric(PIB['VALOR (unidades)'], errors='coerce')
# Usaremos el metodo de LOESS para descomponer el PIB mensualmente 
meses_loees= np.arange(1, 13)
tendencia_mensual_inflacion = [6.94 ,8.01, 8.53 , 9.23 , 9.07 , 9.67 , 10.21 , 10.84 , 11.44, 12.22 , 12.53 , 13.12] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual_inflacion) for valor in tendencia_mensual_inflacion]) 
patron_estacional = patron_estacional / patron_estacional.sum()

# La idea es crear algun tipo de ruido sobre nuestro patron, para asi tratar de simular efectos reales economicos.
np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) #  Asegurarnos que todos las fulctuaciones sean positivas. 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()


def descomponer_pib_loess(pib_anual, patron):
    def asignar_pib_mensual(k, patron):
        pib_mensual = patron * k['VALOR (unidades)']
        return pib_mensual

    df_mensual = pd.DataFrame()
    
    for i, k in pib_anual.iterrows():
        pib_mensual = asignar_pib_mensual(k, patron)
        
        df_temp = pd.DataFrame({
            'Departamento': k['Departamento'],
            'Mes': np.arange(1, 13),
            'PIB_Mensual': pib_mensual
        })
        
        df_mensual = pd.concat([df_mensual, df_temp], ignore_index=True)
    
    return df_mensual

pib_mensual = descomponer_pib_loess(PIB, patron_smooth)
importancia_pesos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\importancia-municipal.xlsx')

#Dado que encontrar una serie completa desde 2018 hasta 2024 de el valor del PIB por ciudad no es facil, primero usaremos la descomposicon por mes que hicimos departamental y la ponderaremos 
#por la importancia de cada ciudad en el departamento para encontrar un aproximando significativo. pdta= la base que nos pasaste profe no tenia todos los anos por eso hacemos esta aproximacion
importancia_pesos = arreglar_texto(importancia_pesos , 'Municipio / Distrito' , 'Ciudad')
importancia_pesos['Departamento'] = importancia_pesos['Departamento'].astype(str)
importancia_pesos = arreglar_texto(importancia_pesos , 'Departamento' , 'Departamento')

# Dejar solo las ciudades que estamos trabajando.
for i in importancia_pesos['Ciudad']:
    indice = importancia_pesos[importancia_pesos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        importancia_pesos = importancia_pesos.drop(index=indice)
    else:
        pass

importancia_pesos_mensual = importancia_pesos.loc[importancia_pesos.index.repeat(12)].reset_index(drop=True)
importancia_pesos_mensual['Mes'] = (importancia_pesos_mensual.index % 12) + 1
pib_pesos_ciudades1 = pd.merge(importancia_pesos_mensual, pib_mensual, on=['Departamento', 'Mes'], how='left')
pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] /100
pib_pesos_ciudades1['Pib Ponderado'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] * pib_pesos_ciudades1['PIB_Mensual']
fusion_left = pd.merge(Base_2018, pib_pesos_ciudades1[['Pib Ponderado' , 'Ciudad', 'Mes']], on=['Ciudad', 'Mes'], how='left')

#Agregar puntos de llegadas internacionales
entradas = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Entradas_de_extranjeros_a_Colombia_20241006.csv')
entradas_2018= entradas[entradas['Año']==2022]
entradas_2018 = entradas_2018[entradas_2018['Latitud - Longitud'] != 'No Aplica,No Aplica']
entradas_2018 =traducir(entradas_2018 , 'Mes')
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )
entradas_2018 =completar_meses(entradas_2018, 'Latitud - Longitud' , 'Mes' , 'Total' ,1 )
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )

entradas_geometry = convertir_ubicacion(entradas_2018 , 'Latitud - Longitud')
Entradas = gpd.GeoDataFrame(entradas_geometry, geometry='geometry')

if Entradas.crs is None:
    Entradas = Entradas.set_crs(Departamentos.crs, allow_override=True)

Entradas = gpd.sjoin_nearest(Entradas,  Departamentos, how='left', distance_col='distancia').drop(['index_right' , 'DPTO' , 'AREA' , 'PERIMETER' , 'HECTARES' ,'distancia'] , axis=1)
Entradas_Departamentos = Entradas.groupby(['Mes', 'Departamento'])['Total'].sum().reset_index()
final = pd.merge(fusion_left, Entradas_Departamentos, on=['Departamento', 'Mes'], how='left').rename(columns={'Total': 'Entradas Extranjeros Zona'})
final['Entradas Extranjeros Zona'] = final['Entradas Extranjeros Zona'].fillna(0)
final[final['Ciudad'] == 'medellin']

#Distancias desde los puntos de llegadas internacionales ponderadas 
for i in ciudades['Ciudad']:
    indice = ciudades[ciudades['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_unico = ciudades.drop(index=indice)
    else:
        pass

ciudades_unico = ciudades_unico.iloc[:, [0, -1]]
ciudades_unico= gpd.GeoDataFrame(ciudades_unico, geometry='geometry')
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')

if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if Migracion_Unico.crs is None:
    Migracion_Unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de Migracion_Unico: {Migracion_Unico.crs}")

ciudades_unico = ciudades_unico.to_crs(epsg=32618)
Migracion_Unico = Migracion_Unico.to_crs(epsg=32618)

ciudades_unico['key'] = 1
Migracion_Unico['key'] = 1

Distancia_combinado = pd.merge(ciudades_unico, Migracion_Unico, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

Distancia_combinado['distancia_km'] = Distancia_combinado.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_combinado['distancia_ponderada'] = Distancia_combinado['distancia_km'] * Distancia_combinado['Total']
Distancia_ponderado = Distancia_combinado.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_migrantes=('Total', 'sum')
).reset_index()

Distancia_ponderado['distancia_ponderada_km'] = Distancia_ponderado['suma_ponderada'] / Distancia_ponderado['suma_migrantes']
Distancia_ponderado_final = Distancia_ponderado[['Ciudad' , 'distancia_ponderada_km']]

Distancia_ciudades = ciudades_unico.merge(Distancia_ponderado_final, on='Ciudad')
final1 = pd.merge(final, Distancia_ciudades, on='Ciudad', how='left').drop(['geometry_y' , 'key'] , axis=1)

# Score importancia puntos entrada
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')
columnas = ['geometry' , 'Departamento' ,'Total' ]
Migracion_Unico_Dept = gpd.sjoin_nearest(Migracion_Unico, Departamentos, how='left', distance_col='distancia')[columnas]
Total_Migracion = Migracion_Unico_Dept.groupby('Departamento').agg(
    puntos=('Departamento', 'size'),  
    total_personas=('Total', 'sum')  
).reset_index()
scaler = MinMaxScaler()
Total_Migracion[['puntos_norm', 'total_norm']] = scaler.fit_transform(Total_Migracion[['puntos', 'total_personas']])
Total_Migracion['importancia accesos'] = 0.3 * Total_Migracion['puntos_norm'] + 0.7 * Total_Migracion['total_norm'] # Asumimos que es mas importante cuanta gente llega a que tantos puntos hay.
final1 = pd.merge(final1, Total_Migracion, on='Departamento', how='left').drop(['puntos' , 'total_personas' , 'puntos_norm' , 'total_norm'] , axis=1)
final1['importancia accesos'] = final1['importancia accesos'].fillna(0)


#Numero establecimientos de turismo, camas y habitaciones para turistas.
hoteles = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Hist_rico_Registro_Nacional_de_Turismo_-_RNT_20241007.csv')
establecimientos_2018 = hoteles[hoteles['AÑO']==2022]
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('PUERTO INIRIDA', 'Inirida', regex=False)
establecimientos_2018['NOMBRE-MUNI'] = establecimientos_2018['NOMBRE-MUNI'].str.replace('BUGA', 'guadalajaradebuga', regex=False)

Establecimientos = establecimientos_2018.groupby('NOMBRE-MUNI').agg({'CATEGORIA': 'count','HABITACIONES': 'sum', 'CAMAS': 'sum'}).reset_index().rename(columns={'CATEGORIA': 'Establecimientos'})
Establecimientos = arreglar_texto(Establecimientos, 'NOMBRE-MUNI' , 'Municipios')
for i in Establecimientos['Municipios']:
    indice = Establecimientos[Establecimientos['Municipios'] == i].index
    if i not in grandes_ciudades:
        Establecimientos = Establecimientos.drop(index=indice)
    else:
        pass

Establecimientos=Establecimientos.rename(columns={'Municipios': 'Ciudad'})
meses_loees = np.arange(1, 13)
tendencia_mensual = [56, 58, 62, 57, 59, 62, 65, 72, 75, 77, 80, 82] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual) for valor in tendencia_mensual]) 
patron_estacional = patron_estacional / patron_estacional.sum()

np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()

def descomponer_variables_loess(df_anual, patron, variables):
    def asignar_valores_mensuales(fila, patron, variables):
        valores_mensuales = {}
        for i in variables:
            valores_mensuales[i] = patron * fila[i]
        return valores_mensuales

    data_mensual = []
    for i, fila in df_anual.iterrows():
        valores_mensuales = asignar_valores_mensuales(fila, patron, variables)
        
        df_temp = pd.DataFrame({
            'Ciudad': fila['Ciudad'],
            'Mes': np.arange(1, 13)
        })
        
        for k in variables:
            df_temp[k] = valores_mensuales[k]
        
        data_mensual.append(df_temp)
    
    df_mensual = pd.concat(data_mensual, ignore_index=True)
    
    return df_mensual

Establecimientos_mensual = descomponer_variables_loess(Establecimientos, patron_smooth , ['Establecimientos' , 'HABITACIONES' , 'CAMAS'])
final2 = pd.merge(final1, Establecimientos_mensual, on=['Ciudad', 'Mes'], how='left')

#Distancias al TOP de colombia 
df_TopColombia = pd.DataFrame.from_dict(Top_Colombia, orient='index', columns=['coordinates','Reviews', 'rating', 'eliminar'])
df_TopColombia = df_TopColombia.reset_index().rename(columns={'index': 'lugar'}).drop('eliminar' , axis=1)
df_TopColombia['geometry'] = df_TopColombia['coordinates'].apply(lambda x: Point(x[1], x[0]))
df_TopColombia_gdp = gpd.GeoDataFrame(df_TopColombia, geometry='geometry').drop('coordinates' , axis=1)
if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if df_TopColombia_gdp.crs is None:
    df_TopColombia_gdp.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de df_TopColombia_gdp: {df_TopColombia_gdp.crs}")


ciudades_unico = ciudades_unico.to_crs(epsg=32618)
df_TopColombia_gdp = df_TopColombia_gdp.to_crs(epsg=32618)

scaler = MinMaxScaler()

ciudades_unico['key'] = 1
df_TopColombia_gdp['key'] = 1

Distancia_Top = pd.merge(ciudades_unico, df_TopColombia_gdp, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

w_reviews = 0.6 # Vamos a asumir que las reviews tienen mas peso que la calificacion del lugar 
w_rating = 0.4

Distancia_Top['distancia_km'] = Distancia_Top.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_Top[['distancia_km','Reviews', 'rating']] = scaler.fit_transform(
    Distancia_Top[['distancia_km','Reviews', 'rating']]
) # Normalizamos las variables para calcular una distancia ponderada con las mismas escalas

Distancia_Top['distancia_ponderada'] = (Distancia_Top['distancia_km']) + ( w_reviews* Distancia_Top['Reviews']) + ( w_rating* Distancia_Top['rating'])

Distancia_Ponderado_Top = Distancia_Top.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_reviews=('Reviews', 'sum'),
    suma_rating=('rating', 'sum')).reset_index()

Distancia_Ponderado_Top['distancia_ponderada_TOP'] = Distancia_Ponderado_Top['suma_ponderada'] / (Distancia_Ponderado_Top['suma_reviews']+Distancia_Ponderado_Top['suma_rating'])
Distancia_Ponderado_Top_final = Distancia_Ponderado_Top[['Ciudad' , 'distancia_ponderada_TOP']]

Distancia_Top = ciudades_unico.merge(Distancia_Ponderado_Top_final, on='Ciudad')
final3 = pd.merge(final2, Distancia_Top, on='Ciudad', how='left').drop(['geometry_x' , 'key'] , axis=1)

# Pobreza con proxy
ciudades_poblacion = ciudades[['Ciudad' , 'population']]
for i in ciudades_poblacion['Ciudad']:
    indice = ciudades_poblacion[ciudades_poblacion['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_poblacion = ciudades_poblacion.drop(index=indice)
    else:
        pass

final4 = pd.merge(final3, ciudades_poblacion, on='Ciudad', how='left')
final4['Proxy_Pobreza'] = final4['Pib Ponderado'] / final4['population']
final4

#Gasto promedio por dia de un turista
gastos_diarios = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\Gastos Diarios 2022.xlsx')
gastos_diarios = arreglar_texto(gastos_diarios , 'Ciudad', 'Ciudad')
ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_diarios = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='left')

gastos_diarios_Nan = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_diarios_Nan = gpd.GeoDataFrame(pd.merge(gastos_diarios_Nan, ciudades_geometry, on='Ciudad', how='left').drop('geometry_x' , axis=1) , geometry='geometry')
gastos_diarios_Nan.set_crs(epsg=4326, inplace=True)
gastos_diarios_Nan=gastos_diarios_Nan.to_crs(epsg=32618)  
variables = [i for i in gastos_diarios_Nan.columns][1:-1]
gastos_sin_Nan = gastos_diarios_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_diarios_Nan[gastos_diarios_Nan[variables].isnull().any(axis=1)]

def kriging(df_con_info, variable, grid_size=100):
    coords = np.array(list(zip(df_con_info.geometry.x, df_con_info.geometry.y)))
    valores = df_con_info[variable].values
    
    min_x, min_y, max_x, max_y = gastos_diarios_Nan.total_bounds
    gridx = np.linspace(min_x, max_x, grid_size)
    gridy = np.linspace(min_y, max_y, grid_size)
    
    Krigg_O = OrdinaryKriging(
        coords[:,0], coords[:,1], valores,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False
    )
    
    z1, ss1 = Krigg_O.execute('grid', gridx, gridy)
    
    return Krigg_O, gridx, gridy, z1

def estimar(Krigg, puntos):
    x = puntos.geometry.x.values
    y = puntos.geometry.y.values
    estimados, ss = Krigg.execute('points', x, y)
    return estimados

for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual['Mes'] = (gastos_krigg_mensual.index % 12) + 1
final5 = pd.merge(final4, gastos_krigg_mensual, on=['Mes', 'Ciudad'], how='left').drop(['geometry_x','population'] , axis=1)

# Gastos de todo el viaje 
gastos_viaje = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2022\\Gastos Viaje 2022.xlsx')
gastos_viaje = arreglar_texto(gastos_viaje , 'Ciudad', 'Ciudad')

ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_viaje = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='left')
gastos_viaje_Nan = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_viaje_Nan = gpd.GeoDataFrame(pd.merge(gastos_viaje_Nan, ciudades_geometry, on='Ciudad', how='left') , geometry='geometry')
gastos_viaje_Nan = gastos_viaje_Nan.drop('geometry_x' , axis=1)

gastos_viaje_Nan.set_crs(epsg=4326, inplace=True)
gastos_viaje_Nan=gastos_viaje_Nan.to_crs(epsg=32618) 
variables = [i for i in gastos_viaje_Nan.columns][1:-1]
gastos_sin_Nan = gastos_viaje_Nan.dropna()
gastos_con_Nan = gastos_viaje_Nan[gastos_viaje_Nan[variables].isnull().any(axis=1)]
for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual_viaje = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual_viaje['Mes'] = (gastos_krigg_mensual_viaje.index % 12) + 1
gastos_krigg_mensual_viaje
Base_2022 = pd.merge(final5, gastos_krigg_mensual_viaje, on=['Mes', 'Ciudad'], how='left').drop('geometry_y' , axis=1)

# Agregar inflacion
inflacion = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Inflacion.xlsx')
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion[inflacion['Fecha'].dt.year == 2022].rename(columns={'Fecha':'Mes'})
inflacion['Mes'] = inflacion['Mes'].dt.month
Base_2022  = pd.merge(Base_2022 , inflacion, on='Mes', how='left')

#Agregar Cantidad de vias 
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2022 = pd.merge(Base_2022 , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

#Agregar Vias
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2022 = pd.merge(Base_2022 , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

# Agregar Eventos
eventos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Eventos Turisticos\\Eventos 2023.xlsx').fillna(0)
eventos = arreglar_texto(eventos ,'Departamento' , 'Departamento')
eventos['Departamento'] = eventos['Departamento'].str.replace('sanandres', 'archipielagodesanandresprovidenciaysantacatalina', regex=False)
Base_2022 = pd.merge(Base_2022 , eventos , on=['Departamento' ,'Mes'] , how='left')

## Lets see 2022 DataFrame 

In [21]:
Base_2022 = Base_2022.drop('carreteras_cercanas_y' , axis=1)
Base_2022

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas_x,Eventos
0,ipiales,1,662.0,2,106,16,narino,21.19639,3987.32,107.664827,...,32859.977774,414930.568037,85339.410173,81456.596608,135766.83125,166329.823144,POINT (205670.004 91871.64),6.94460,5,1.0
1,sogamoso,1,64.0,0,67,5,boyaca,19.50000,3987.32,224.009782,...,36557.359071,432151.277957,83777.318179,86829.017931,135766.83125,146312.971893,POINT (730269.76 632304.448),6.94460,6,2.0
2,villavicencio,1,369.0,3,375,35,meta,26.85000,3987.32,957.694236,...,25404.702000,520055.783000,82656.411000,146138.129000,130711.24400,160549.999000,POINT (651697.732 458838.689),6.94460,11,4.0
3,cali,1,10522.0,74,1414,72,valledelcauca,15.40000,3987.32,3799.338865,...,46615.889000,684311.314000,161859.618000,78320.118000,193990.36800,250141.209000,POINT (330897.116 378217.634),6.94460,11,1.0
4,pasto,1,412.0,4,317,41,narino,11.40000,3987.32,414.182292,...,29348.193000,367035.791000,54313.047000,76923.127000,95936.82900,139862.787000,POINT (246593.462 133604.578),6.94460,9,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,piedecuesta,12,137.0,3,106,2,santander,21.60000,4780.17,414.508319,...,25463.461156,517058.891202,107775.602005,92543.117839,135766.83125,135485.158165,POINT (720906.443 783432.99),13.12247,4,0.0
968,cartago,12,524.0,10,45,4,valledelcauca,22.45000,4780.17,377.728402,...,42700.702452,445954.853897,81813.678105,71490.134752,135766.83125,193287.643915,POINT (398330.654 519569.549),13.12247,10,2.0
969,uribia,12,5.0,2,8,1,laguajira,26.20000,4780.17,148.087212,...,54748.566907,729532.882737,88332.548939,138476.840738,135766.83125,344938.481437,POINT (826814.742 1319110.428),13.12247,0,1.0
970,inirida,12,13.0,0,7,2,guainia,21.20039,4780.17,42.175835,...,52419.185547,626627.417683,88332.548939,99990.243989,135766.83125,309797.236065,POINT (1287613.509 430529.596),13.12247,0,0.0


In [22]:

Base_2022[Base_2022.isna().any(axis=1)]

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas_x,Eventos


# 2023

In [ ]:
#Inicio proceso
df= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Visitantes_No_Residentes.csv')
df1 = df[df["Año"] == 2023]
df1= arreglar_texto(df1 , 'Ciudad' , 'Ciudad')
Homicidios= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\homicidio intencional.xlsx').dropna()
Homicidos1 = bases_crimenes(Homicidios , 'FECHA HECHO' , 'CANTIDAD' , 'Homicidios')
Hurto= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\hurto a personas.xlsx').dropna()
Hurto1 = bases_crimenes(Hurto , 'FECHA HECHO' , 'CANTIDAD' , 'Hurtos')
sexuales= pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\delitos sexuales.xlsx').dropna()
sexuales1 = bases_crimenes(sexuales , 'FECHA HECHO' , 'CANTIDAD' , 'Delitos Sexuales')
ciudades= pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\ciudades.csv')
ciudades = arreglar_texto(ciudades, 'city' , "Ciudad")
ciudades['admin_name'] = ciudades['admin_name'].astype(str)
ciudades.at[68, 'admin_name'] = 'Atlantico'
Total_18= completar_meses(df1 , 'Ciudad' , 'Mes' ,'Extranjeros no Residentes' , 1)
Total_18 = pd.merge(Total_18, Homicidos1[['Ciudad', 'Homicidios' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, Hurto1[['Ciudad', 'Hurtos' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18 = pd.merge(Total_18, sexuales1[['Ciudad', 'Delitos Sexuales' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18['Homicidios'] = Total_18['Homicidios'].fillna(0)
Total_18['Hurtos'] = Total_18['Hurtos'].fillna(0)
Total_18['Delitos Sexuales'] = Total_18['Delitos Sexuales'].fillna(0)
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('bogotadc', 'bogota', regex=False)

#Seleccionar Ciudades que vamos a trabajar
ciudades1= set(ciudades['Ciudad'])
ciudades2= set(Total_18['Ciudad'])
grandes_ciudades=list(ciudades1.intersection(ciudades2))
l=['bogota' , 'tumaco']
grandes_ciudades = grandes_ciudades + l
Total_18['Ciudad'] = Total_18['Ciudad'].str.replace('sanandresdetumaco', 'tumaco', regex=False)
for i in Total_18['Ciudad']:
    indice = Total_18[Total_18['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Total_18 = Total_18.drop(index=indice)
    else:
        pass

#Poner Ubicaciones
ciudades['geometry'] = ciudades.apply(lambda i: Point(i['lng'], i['lat']), axis=1)
ciudades_geometry= gpd.GeoDataFrame(ciudades, geometry='geometry')
Total_18_Geometry= pd.merge(Total_18, ciudades_geometry[['Ciudad', 'geometry']], on='Ciudad' , how='left')
Total_18_Geometry= gpd.GeoDataFrame(Total_18_Geometry, geometry='geometry')

# Poner departamentos
Departamentos = gpd.read_file("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Colombia.json")
Departamentos = arreglar_texto(Departamentos, 'NOMBRE_DPT' , "Departamento")
Departamentos["Departamento"] = Departamentos["Departamento"].str.replace('santafedebogota', 'bogotadc', regex=False)

if Total_18_Geometry.crs is None:
    Total_18_Geometry = Total_18_Geometry.set_crs(Departamentos.crs, allow_override=True)

Total_18_Geometry = gpd.sjoin_nearest(Total_18_Geometry,  Departamentos, how='left', distance_col='distancia').drop(['AREA' , 'PERIMETER' , 'HECTARES' , 'DPTO','index_right' , 'distancia'] , axis=1)
Total_18_Geometry
Total_18_Geometry[Total_18_Geometry.isnull().any(axis=1)]

#Agregar Clima
clima2=pd.read_csv("C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\Datos_Hidrometeorol_gicos_Crudos_-_Red_de_Estaciones_IDEAM___Temperatura_20241009.csv")
clima2['FechaObservacion'] = pd.to_datetime(clima2['FechaObservacion'])
clima2['Mes'] = clima2['FechaObservacion'].dt.month_name()

clima_2018 = clima2[clima2['FechaObservacion'].dt.year == 2023]
clima_2018 = arreglar_texto(clima_2018 , 'Municipio', 'Ciudad')
clima_2018= completar_meses(clima_2018 , 'Ciudad' , 'Mes' ,'ValorObservado' , 0)
Total_18_Clima= pd.merge(Total_18_Geometry, clima_2018[['Ciudad', 'ValorObservado' ,'Mes']], on=['Ciudad' ,'Mes'], how='left')
Total_18_Clima = Total_18_Clima.rename(columns={'ValorObservado': 'Temperatura'})

#Krigging para el clima 
df_kriging = meses_a_numeros(Total_18_Clima , 'Mes')
df_kriging.set_crs(epsg=4326, inplace=True)
df_kriging=df_kriging.to_crs(epsg=32618)  

meses_krig = df_kriging['Mes'].unique()

for i in meses_krig:
    df_kriging_mes = df_kriging[df_kriging['Mes'] == i] # Vamos a realizar el kriging por mes
    df_kriging_known = df_kriging_mes.dropna(subset=['Temperatura']) # Los datos para los que tenemos temperatura
    df_kriging_missing = df_kriging_mes[df_kriging_mes ['Temperatura'].isna()] # Los datos para los NO que tenemos temperatura (Variable a predecir)
    
    if df_kriging_missing.empty:
        print('No hay datos faltantes en el mes ' , i) # Si algun mes esta completo para todas las ciudades, no hacer nada 
        continue
    
    # Nuestras variables para predecir
    x_known = df_kriging_known.geometry.x.values
    y_known = df_kriging_known.geometry.y.values
    z_known = df_kriging_known['Temperatura'].values
    
    # Nuestros datos a predecir
    x_missing = df_kriging_missing.geometry.x.values
    y_missing = df_kriging_missing.geometry.y.values
    
    # Crear el modelo 
    Krigg = OrdinaryKriging(
        x_known, y_known, z_known,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False)
    
    # Realizar predicciones con el krigging
    z_pred, ss = Krigg.execute('points', x_missing, y_missing)
    
    # Introducir en el DF los valores
    df_kriging.loc[df_kriging_missing.index, 'Temperatura'] = z_pred

#Agregar precio dolar
Dolar = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Dolar.xlsx')
Dolar['Fecha'] = pd.to_datetime(Dolar['Fecha'], errors='coerce')
Dolar.set_index('Fecha', inplace=True)
Dolar = Dolar.resample('M').median()
Dolar = Dolar.reset_index()
Dolar_2018=Dolar[Dolar['Fecha'].dt.year == 2023]
Dolar_2018['Mes'] = range(1, 13)
Base_2018=pd.merge(df_kriging, Dolar_2018, on= 'Mes', how='right').drop('Fecha' , axis=1)
Base_2018

#Interpolar PIB mensual

PIB = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\PIB - Miles de millones de pesos  - 2023pr.xlsx')
PIB= arreglar_texto(PIB, 'DEPARTAMENTO' , 'Departamento')
PIB['VALOR (unidades)'] = PIB['VALOR (unidades)'].str.replace(',', '.')
PIB['VALOR (unidades)'] = pd.to_numeric(PIB['VALOR (unidades)'], errors='coerce')
# Usaremos el metodo de LOESS para descomponer el PIB mensualmente 
meses_loees= np.arange(1, 13)
tendencia_mensual_inflacion = [13.25 ,13.28, 13.34 , 12.82 , 12.36 , 12.13 , 11.78 , 11.43 , 10.09, 10.48 , 10.15 , 9.28] # Esta tendencia se puede ver en la imagen 
patron_estacional = np.array([valor / sum(tendencia_mensual_inflacion) for valor in tendencia_mensual_inflacion]) 
patron_estacional = patron_estacional / patron_estacional.sum()

# La idea es crear algun tipo de ruido sobre nuestro patron, para asi tratar de simular efectos reales economicos.
np.random.seed(42)
patron_ruido = patron_estacional + np.random.normal(0, 0.005, 12)
patron_ruido = np.clip(patron_ruido, 0.01, None) #  Asegurarnos que todos las fulctuaciones sean positivas. 
patron_ruido = patron_ruido / patron_ruido.sum()

loess_result = lowess(patron_ruido, meses_loees, frac=0.4)
meses_smooth = loess_result[:, 0]
patron_smooth = loess_result[:, 1]
patron_smooth = patron_smooth / patron_smooth.sum()


def descomponer_pib_loess(pib_anual, patron):
    def asignar_pib_mensual(k, patron):
        pib_mensual = patron * k['VALOR (unidades)']
        return pib_mensual

    df_mensual = pd.DataFrame()
    
    for i, k in pib_anual.iterrows():
        pib_mensual = asignar_pib_mensual(k, patron)
        
        df_temp = pd.DataFrame({
            'Departamento': k['Departamento'],
            'Mes': np.arange(1, 13),
            'PIB_Mensual': pib_mensual
        })
        
        df_mensual = pd.concat([df_mensual, df_temp], ignore_index=True)
    
    return df_mensual

pib_mensual = descomponer_pib_loess(PIB, patron_smooth)
importancia_pesos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\importancia-municipal.xlsx')

#Dado que encontrar una serie completa desde 2018 hasta 2024 de el valor del PIB por ciudad no es facil, primero usaremos la descomposicon por mes que hicimos departamental y la ponderaremos 
#por la importancia de cada ciudad en el departamento para encontrar un aproximando significativo. pdta= la base que nos pasaste profe no tenia todos los anos por eso hacemos esta aproximacion
importancia_pesos = arreglar_texto(importancia_pesos , 'Municipio / Distrito' , 'Ciudad')
importancia_pesos['Departamento'] = importancia_pesos['Departamento'].astype(str)
importancia_pesos = arreglar_texto(importancia_pesos , 'Departamento' , 'Departamento')

# Dejar solo las ciudades que estamos trabajando.
for i in importancia_pesos['Ciudad']:
    indice = importancia_pesos[importancia_pesos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        importancia_pesos = importancia_pesos.drop(index=indice)
    else:
        pass

importancia_pesos_mensual = importancia_pesos.loc[importancia_pesos.index.repeat(12)].reset_index(drop=True)
importancia_pesos_mensual['Mes'] = (importancia_pesos_mensual.index % 12) + 1
pib_pesos_ciudades1 = pd.merge(importancia_pesos_mensual, pib_mensual, on=['Departamento', 'Mes'], how='left')
pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] /100
pib_pesos_ciudades1['Pib Ponderado'] = pib_pesos_ciudades1['Peso relativo municipal en el valor agregado departamental %'] * pib_pesos_ciudades1['PIB_Mensual']
fusion_left = pd.merge(Base_2018, pib_pesos_ciudades1[['Pib Ponderado' , 'Ciudad', 'Mes']], on=['Ciudad', 'Mes'], how='left')

#Agregar puntos de llegadas internacionales
entradas = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Entradas_de_extranjeros_a_Colombia_20241006.csv')
entradas_2018= entradas[entradas['Año']==2022]
entradas_2018 = entradas_2018[entradas_2018['Latitud - Longitud'] != 'No Aplica,No Aplica']
entradas_2018 =traducir(entradas_2018 , 'Mes')
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )
entradas_2018 =completar_meses(entradas_2018, 'Latitud - Longitud' , 'Mes' , 'Total' ,1 )
entradas_2018 =meses_a_numeros(entradas_2018 , 'Mes' )

entradas_geometry = convertir_ubicacion(entradas_2018 , 'Latitud - Longitud')
Entradas = gpd.GeoDataFrame(entradas_geometry, geometry='geometry')

if Entradas.crs is None:
    Entradas = Entradas.set_crs(Departamentos.crs, allow_override=True)

Entradas = gpd.sjoin_nearest(Entradas,  Departamentos, how='left', distance_col='distancia').drop(['index_right' , 'DPTO' , 'AREA' , 'PERIMETER' , 'HECTARES' ,'distancia'] , axis=1)
Entradas_Departamentos = Entradas.groupby(['Mes', 'Departamento'])['Total'].sum().reset_index()
final = pd.merge(fusion_left, Entradas_Departamentos, on=['Departamento', 'Mes'], how='left').rename(columns={'Total': 'Entradas Extranjeros Zona'})
final['Entradas Extranjeros Zona'] = final['Entradas Extranjeros Zona'].fillna(0)

#Distancias desde los puntos de llegadas internacionales ponderadas 
for i in ciudades['Ciudad']:
    indice = ciudades[ciudades['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_unico = ciudades.drop(index=indice)
    else:
        pass

ciudades_unico = ciudades_unico.iloc[:, [0, -1]]
ciudades_unico= gpd.GeoDataFrame(ciudades_unico, geometry='geometry')
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')

if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if Migracion_Unico.crs is None:
    Migracion_Unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de Migracion_Unico: {Migracion_Unico.crs}")

ciudades_unico = ciudades_unico.to_crs(epsg=32618)
Migracion_Unico = Migracion_Unico.to_crs(epsg=32618)

ciudades_unico['key'] = 1
Migracion_Unico['key'] = 1

Distancia_combinado = pd.merge(ciudades_unico, Migracion_Unico, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

Distancia_combinado['distancia_km'] = Distancia_combinado.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_combinado['distancia_ponderada'] = Distancia_combinado['distancia_km'] * Distancia_combinado['Total']
Distancia_ponderado = Distancia_combinado.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_migrantes=('Total', 'sum')
).reset_index()

Distancia_ponderado['distancia_ponderada_km'] = Distancia_ponderado['suma_ponderada'] / Distancia_ponderado['suma_migrantes']
Distancia_ponderado_final = Distancia_ponderado[['Ciudad' , 'distancia_ponderada_km']]

Distancia_ciudades = ciudades_unico.merge(Distancia_ponderado_final, on='Ciudad')
final1 = pd.merge(final, Distancia_ciudades, on='Ciudad', how='left').drop(['geometry_y' , 'key'] , axis=1)

# Score importancia puntos entrada
Migracion_Unico = entradas_2018.groupby('Latitud - Longitud')['Total'].sum().reset_index().iloc[:-1, :]
Migracion_Unico = convertir_ubicacion(Migracion_Unico , 'Latitud - Longitud')
Migracion_Unico = gpd.GeoDataFrame(Migracion_Unico, geometry='geometry')
columnas = ['geometry' , 'Departamento' ,'Total' ]
Migracion_Unico_Dept = gpd.sjoin_nearest(Migracion_Unico, Departamentos, how='left', distance_col='distancia')[columnas]
Total_Migracion = Migracion_Unico_Dept.groupby('Departamento').agg(
    puntos=('Departamento', 'size'),  
    total_personas=('Total', 'sum')  
).reset_index()
scaler = MinMaxScaler()
Total_Migracion[['puntos_norm', 'total_norm']] = scaler.fit_transform(Total_Migracion[['puntos', 'total_personas']])
Total_Migracion['importancia accesos'] = 0.3 * Total_Migracion['puntos_norm'] + 0.7 * Total_Migracion['total_norm'] # Asumimos que es mas importante cuanta gente llega a que tantos puntos hay.
final1 = pd.merge(final1, Total_Migracion, on='Departamento', how='left').drop(['puntos' , 'total_personas' , 'puntos_norm' , 'total_norm'] , axis=1)
final1['importancia accesos'] = final1['importancia accesos'].fillna(0)


#Numero establecimientos de turismo, camas y habitaciones para turistas.
hoteles = pd.read_csv('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\Prestadores_de_servicios_Tur_sticos_20241009.csv')
hoteles['MUNICIPIO'] = hoteles['MUNICIPIO'].str.replace('PUERTO INIRIDA', 'Inirida', regex=False)
hoteles['MUNICIPIO'] = hoteles['MUNICIPIO'].str.replace('BUGA', 'guadalajaradebuga', regex=False)

Establecimientos = hoteles.groupby(['MUNICIPIO' , 'MES']).agg({'CATEGORIA': 'count','NÚMERO DE HABIATACIONES': 'sum', 'NÚMERO DE CAMAS': 'sum'}).reset_index().rename(columns={'CATEGORIA': 'Establecimientos'})
Establecimientos = arreglar_texto(Establecimientos, 'MUNICIPIO' , 'Municipios')

Establecimientos=Establecimientos.rename(columns={'Municipios': 'Ciudad'})
Establecimientos=Establecimientos.rename(columns={'MES': 'Mes'})

for i in Establecimientos['Ciudad']:
    indice = Establecimientos[Establecimientos['Ciudad'] == i].index
    if i not in grandes_ciudades:
        Establecimientos = Establecimientos.drop(index=indice)
    else:
        pass

ciudades_unicas = Establecimientos['Ciudad'].unique()
meses_completos = pd.DataFrame({'Mes': range(1, 13)})
Establecimientos_completo = pd.MultiIndex.from_product([ciudades_unicas, range(1, 13)], names=['Ciudad', 'Mes']).to_frame(index=False)
Establecimientos_completo = pd.merge(Establecimientos_completo, Establecimientos , how='outer', on=['Ciudad', 'Mes'])
Establecimientos_completo 

final2 = pd.merge(final1, Establecimientos_completo  , on=['Ciudad', 'Mes'], how='left')

#Distancias al TOP de colombia 
df_TopColombia = pd.DataFrame.from_dict(Top_Colombia, orient='index', columns=['coordinates','Reviews', 'rating', 'eliminar'])
df_TopColombia = df_TopColombia.reset_index().rename(columns={'index': 'lugar'}).drop('eliminar' , axis=1)
df_TopColombia['geometry'] = df_TopColombia['coordinates'].apply(lambda x: Point(x[1], x[0]))
df_TopColombia_gdp = gpd.GeoDataFrame(df_TopColombia, geometry='geometry').drop('coordinates' , axis=1)
if ciudades_unico.crs is None:
    ciudades_unico.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de ciudades_unico: {ciudades_unico.crs}")

if df_TopColombia_gdp.crs is None:
    df_TopColombia_gdp.set_crs(epsg=4326, inplace=True)
else:
    print(f"CRS de df_TopColombia_gdp: {df_TopColombia_gdp.crs}")


ciudades_unico = ciudades_unico.to_crs(epsg=32618)
df_TopColombia_gdp = df_TopColombia_gdp.to_crs(epsg=32618)

scaler = MinMaxScaler()

ciudades_unico['key'] = 1
df_TopColombia_gdp['key'] = 1

Distancia_Top = pd.merge(ciudades_unico, df_TopColombia_gdp, on='key', suffixes=('_ciudad', '_punto')).drop('key', axis=1)

w_reviews = 0.6 # Vamos a asumir que las reviews tienen mas peso que la calificacion del lugar 
w_rating = 0.4

Distancia_Top['distancia_km'] = Distancia_Top.apply(lambda x: x['geometry_punto'].distance(x['geometry_ciudad']) / 1000, axis=1) # Calcular la distancia 

Distancia_Top[['distancia_km','Reviews', 'rating']] = scaler.fit_transform(
    Distancia_Top[['distancia_km','Reviews', 'rating']]
) # Normalizamos las variables para calcular una distancia ponderada con las mismas escalas

Distancia_Top['distancia_ponderada'] = (Distancia_Top['distancia_km']) + ( w_reviews* Distancia_Top['Reviews']) + ( w_rating* Distancia_Top['rating'])

Distancia_Ponderado_Top = Distancia_Top.groupby('Ciudad').agg(
    suma_ponderada=('distancia_ponderada', 'sum'),
    suma_reviews=('Reviews', 'sum'),
    suma_rating=('rating', 'sum')).reset_index()

Distancia_Ponderado_Top['distancia_ponderada_TOP'] = Distancia_Ponderado_Top['suma_ponderada'] / (Distancia_Ponderado_Top['suma_reviews']+Distancia_Ponderado_Top['suma_rating'])
Distancia_Ponderado_Top_final = Distancia_Ponderado_Top[['Ciudad' , 'distancia_ponderada_TOP']]

Distancia_Top = ciudades_unico.merge(Distancia_Ponderado_Top_final, on='Ciudad')
final3 = pd.merge(final2, Distancia_Top, on='Ciudad', how='left').drop(['geometry_x' , 'key'] , axis=1)

# Pobreza con proxy
ciudades_poblacion = ciudades[['Ciudad' , 'population']]
for i in ciudades_poblacion['Ciudad']:
    indice = ciudades_poblacion[ciudades_poblacion['Ciudad'] == i].index
    if i not in grandes_ciudades:
        ciudades_poblacion = ciudades_poblacion.drop(index=indice)
    else:
        pass

final4 = pd.merge(final3, ciudades_poblacion, on='Ciudad', how='left')
final4['Proxy_Pobreza'] = final4['Pib Ponderado'] / final4['population']
final4

#Gasto promedio por dia de un turista
gastos_diarios = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\Gasto Diario 2023.xlsx')
gastos_diarios = arreglar_texto(gastos_diarios , 'Ciudad', 'Ciudad')
ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_diarios = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='left')

gastos_diarios_Nan = pd.merge(gastos_diarios, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_diarios_Nan = gpd.GeoDataFrame(pd.merge(gastos_diarios_Nan, ciudades_geometry, on='Ciudad', how='left').drop('geometry_x' , axis=1) , geometry='geometry')
gastos_diarios_Nan.set_crs(epsg=4326, inplace=True)
gastos_diarios_Nan=gastos_diarios_Nan.to_crs(epsg=32618)  
variables = [i for i in gastos_diarios_Nan.columns][1:-1]
gastos_sin_Nan = gastos_diarios_Nan.dropna(subset=variables)
gastos_con_Nan = gastos_diarios_Nan[gastos_diarios_Nan[variables].isnull().any(axis=1)]

def kriging(df_con_info, variable, grid_size=100):
    coords = np.array(list(zip(df_con_info.geometry.x, df_con_info.geometry.y)))
    valores = df_con_info[variable].values
    
    min_x, min_y, max_x, max_y = gastos_diarios_Nan.total_bounds
    gridx = np.linspace(min_x, max_x, grid_size)
    gridy = np.linspace(min_y, max_y, grid_size)
    
    Krigg_O = OrdinaryKriging(
        coords[:,0], coords[:,1], valores,
        variogram_model='spherical',
        verbose=False,
        enable_plotting=False
    )
    
    z1, ss1 = Krigg_O.execute('grid', gridx, gridy)
    
    return Krigg_O, gridx, gridy, z1

def estimar(Krigg, puntos):
    x = puntos.geometry.x.values
    y = puntos.geometry.y.values
    estimados, ss = Krigg.execute('points', x, y)
    return estimados

for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual['Mes'] = (gastos_krigg_mensual.index % 12) + 1
final5 = pd.merge(final4, gastos_krigg_mensual, on=['Mes', 'Ciudad'], how='left').drop(['geometry_x','population'] , axis=1)

# Gastos de todo el viaje 
gastos_viaje = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Bases 2023\\Gastos Viaje.xlsx')
gastos_viaje = arreglar_texto(gastos_viaje , 'Ciudad', 'Ciudad')

ciudades_geometry = Total_18_Geometry[['Ciudad' , 'geometry']].drop_duplicates()
gastos_viaje = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='left')
gastos_viaje_Nan = pd.merge(gastos_viaje, ciudades_geometry, on='Ciudad', how='outer').drop('geometry_y' , axis=1)
gastos_viaje_Nan = gpd.GeoDataFrame(pd.merge(gastos_viaje_Nan, ciudades_geometry, on='Ciudad', how='left') , geometry='geometry')
gastos_viaje_Nan = gastos_viaje_Nan.drop('geometry_x' , axis=1)

gastos_viaje_Nan.set_crs(epsg=4326, inplace=True)
gastos_viaje_Nan=gastos_viaje_Nan.to_crs(epsg=32618) 
variables = [i for i in gastos_viaje_Nan.columns][1:-1]
gastos_sin_Nan = gastos_viaje_Nan.dropna()
gastos_con_Nan = gastos_viaje_Nan[gastos_viaje_Nan[variables].isnull().any(axis=1)]
for var in variables:
    OK, gridx, gridy, z1 = kriging(gastos_sin_Nan, var)
    
    estimados = estimar(OK, gastos_con_Nan)
    gastos_con_Nan[var] = estimados

gastos_krigg = pd.concat([gastos_con_Nan, gastos_sin_Nan], ignore_index=True)
gastos_krigg_mensual_viaje = gastos_krigg.loc[gastos_krigg.index.repeat(12)].reset_index(drop=True)
gastos_krigg_mensual_viaje['Mes'] = (gastos_krigg_mensual_viaje.index % 12) + 1
gastos_krigg_mensual_viaje
Base_2023 = pd.merge(final5, gastos_krigg_mensual_viaje, on=['Mes', 'Ciudad'], how='left').drop('geometry_y' , axis=1)

# Agregar inflacion
inflacion = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Inflacion.xlsx')
inflacion['Fecha'] = pd.to_datetime(inflacion['Fecha'])
inflacion = inflacion[inflacion['Fecha'].dt.year == 2023].rename(columns={'Fecha':'Mes'})
inflacion['Mes'] = inflacion['Mes'].dt.month
Base_2023  = pd.merge(Base_2023  , inflacion, on='Mes', how='left')


#Agregar Cantidad de vias 
vias = gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\RedVial.zip')
vias['distancia'] = abs(vias['distanciaf'] - vias['distanciai'])
vias_imp = vias[['distancia' , 'geometry']]
ciudades_geom= gpd.read_file('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Ciudades Colombia Geometry.geojson')
ciudades_geom = ciudades_geom.to_crs(epsg=4326)
vias_imp = vias_imp.to_crs(epsg=4326)
ciudades_geom = ciudades_geom.to_crs(epsg=32617)
vias_imp = vias_imp.to_crs(epsg=32617)

buffer_distancia = 10000 

ciudades_geom['buffer'] = ciudades_geom.geometry.buffer(buffer_distancia)
ciudades_buffer = gpd.GeoDataFrame(ciudades_geom, geometry='buffer', crs=ciudades_geom.crs).drop('geometry' , axis=1)

join = gpd.sjoin(vias_imp, ciudades_buffer, how='inner', predicate='intersects')
conteo_carreteras = join.groupby('Ciudades').size().reset_index(name='carreteras_cercanas')
ciudades_geom = ciudades_geom.merge(conteo_carreteras, on='Ciudades', how='left')
ciudades_geom['carreteras_cercanas'] = ciudades_geom['carreteras_cercanas'].fillna(0).astype(int)
ciudades_geom =  arreglar_texto(ciudades_geom, 'Ciudades' , 'Ciudad')

Base_2023 = pd.merge(Base_2023 , ciudades_geom[['Ciudad' , 'carreteras_cercanas']] , on='Ciudad' , how='left')

# Agregar Eventos
eventos = pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Eventos Turisticos\\Eventos 2023.xlsx').fillna(0)
eventos = arreglar_texto(eventos ,'Departamento' , 'Departamento')
eventos['Departamento'] = eventos['Departamento'].str.replace('sanandres', 'archipielagodesanandresprovidenciaysantacatalina', regex=False)
Base_2023 = pd.merge(Base_2023 , eventos , on=['Departamento' ,'Mes'] , how='left')

## Lets see 2023 DataFrame 

In [24]:
Base_2023

,Ciudad,Mes,Extranjeros no Residentes,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Otros Gastos Diario,Gasto Promedio Viaje,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,geometry,Inflacion,carreteras_cercanas,Eventos
0,envigado,1,1180.0,0,119,4,antioquia,21.314463,4692.04,1315.923830,...,41115.318939,469234.311565,85042.863721,76311.721879,146377.089587,165951.596867,POINT (437301.941 681665.313),13.25198,6,3.0
1,pereira,1,3032.0,5,349,27,risaralda,19.700000,4692.04,1495.867681,...,28310.677000,300839.919000,64017.664000,54227.832000,78897.842000,103696.581000,POINT (422977.411 532176.544),13.25198,14,0.0
2,malambo,1,11.0,4,38,4,atlantico,21.314463,4692.04,317.118690,...,36597.729686,474340.266423,92645.185569,94965.322047,144763.300117,187671.794038,POINT (527324.757 1199405.166),13.25198,8,0.0
3,bucaramanga,1,1966.0,11,833,21,santander,21.700000,4692.04,2893.910694,...,31245.180000,485356.241000,90652.926000,86058.169000,155661.876000,152983.271000,POINT (720882.544 788963.63),13.25198,9,3.0
4,armenia,1,1262.0,7,194,12,quindio,19.800000,4692.04,687.543890,...,21342.029000,346819.887000,77283.026000,84550.779000,88250.654000,96735.428000,POINT (424566.064 500746.928),13.25198,11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,inirida,12,9.0,0,5,2,guainia,26.650000,3956.76,28.530179,...,66570.629446,775630.771711,96724.213292,80971.122105,148406.448920,398169.400513,POINT (1287613.509 430529.596),9.27605,0,0.0
968,lajaguadeibirico,12,49.0,1,7,0,cesar,24.466558,3956.76,417.451225,...,29560.223758,551409.688748,96724.213292,115538.308052,147201.698640,180503.209527,POINT (682924.172 1057948.201),9.27605,1,0.0
969,quibdo,12,28.0,5,72,7,choco,23.795814,3956.76,131.489163,...,37083.408000,576955.167000,32778.482000,98407.282000,165761.578000,280007.826000,POINT (316373.42 629443.463),9.27605,2,0.0
970,mitu,12,1.0,0,3,3,vaupes,23.726524,3956.76,16.948123,...,58859.141274,634676.114547,96724.213292,81662.941349,144525.028720,336723.130284,POINT (1037614.182 132922.769),9.27605,0,0.0


In [25]:
Base_2023.columns[Base_2023 .isna().any()].tolist()

['Establecimientos', 'NÚMERO DE HABIATACIONES', 'NÚMERO DE CAMAS']

# Combine all years by adding the image data.

In [33]:
Base_2018_Final = Base_2018_Final.drop('geometry_x' , axis=1)

In [34]:
Columnas_Nombres=['Ciudad', 'Mes', 'Nmero Extranjeros', 'Homicidios', 'Hurtos',
       'Delitos Sexuales', 'Departamento', 'Temperatura', 'Dolar',
       'Pib Ponderado', 'Entradas Extranjeros Zona', 'Distancia a accseos','importancia accesos',
       'Establecimientos de turismo', 'N Habitaciones', 'N Camas', 'Distancia al TOP',
       'Proxy Pobreza', 'Gasto Promedio Diario', 'Gasto Alojamiento Diario',
       'Gasto Transporte Diario', 'Gasto alimetos Diario',
       'Otros Gastos Diario', 'Gasto Promedio Viaje',
       'Gasto  Alojamiento Viaje', 'Gasto Transporte Viaje',
       'Gasto alimetos Viaje', 'Otros Gastos Viaje' ,'geometry', 'Inflacion' , 'Nmero Vias' , 'Eventos']

bases=[Base_2018_Final, Base_2019, Base_2021, Base_2022 , Base_2023]

for i in bases:
       i.columns=Columnas_Nombres
       if any(i[col].tolist() == 'geometry' for col in i.columns):
              i.drop(columns=['geometry'], inplace=True)
       else:
              pass


## We load the image data to include them in the database


In [36]:

datos_imagenes=pd.read_excel('C:\\Users\\alejo\\OneDrive\\Escritorio\\Universidaad\\Colab Notebooks\\Ia en economia\\Proyecto\\Areas_Finales.xlsx')
datos_imagenes= arreglar_texto(datos_imagenes , 'ciudad' , 'Ciudad')

La funcion demoro 0.0026547908782958984


In [37]:
def agregar_imagenes(df , imagenes , c):
    imagenes = imagenes[imagenes['año'] ==c]
    df1 = pd.merge(df , imagenes, on='Ciudad' , how='left')
    return df1

Base_2018_img = agregar_imagenes(Base_2018_Final , datos_imagenes , 2018)
Base_2019_img = agregar_imagenes(Base_2019 , datos_imagenes , 2019)
Base_2021_img = agregar_imagenes(Base_2021 , datos_imagenes , 2021)
Base_2022_img = agregar_imagenes(Base_2022 , datos_imagenes , 2022)
Base_2023_img = agregar_imagenes(Base_2023 , datos_imagenes , 2023)

Base_2018_img['Mes'] = Base_2018_img['Mes'].astype(str).str.cat(['-2018']*len(Base_2018_img), sep='')
Base_2019_img['Mes'] = Base_2019_img['Mes'].astype(str).str.cat(['-2019']*len(Base_2019_img), sep='')
Base_2021_img['Mes'] = Base_2021_img['Mes'].astype(str).str.cat(['-2021']*len(Base_2021_img), sep='')
Base_2022_img['Mes'] = Base_2022_img['Mes'].astype(str).str.cat(['-2022']*len(Base_2022_img), sep='')
Base_2023_img['Mes'] = Base_2023_img['Mes'].astype(str).str.cat(['-2023']*len(Base_2023_img), sep='')

In [38]:
Base_Turismo_PD = pd.concat([Base_2018_img, Base_2019_img, Base_2021_img, Base_2022_img , Base_2023_img], ignore_index=True).drop(['año' ,'geometry'] , axis=1)

## Final database with missing data

In [39]:

Base_Turismo_PD

,Ciudad,Mes,Nmero Extranjeros,Homicidios,Hurtos,Delitos Sexuales,Departamento,Temperatura,Dolar,Pib Ponderado,...,Gasto Alojamiento Viaje,Gasto Transporte Viaje,Gasto alimetos Viaje,Otros Gastos Viaje,Inflacion,Nmero Vias,Eventos,Area Urbana,Area Rural,Area Agua
0,pitalito,1-2018,33.0,2,61,15,huila,18.055550,2855.86,151.999238,...,51427.415415,83910.734197,86719.142328,129164.039202,3.679528,6,0.0,2.793593,0.818258,0.005400
1,riohacha,1-2018,108.0,4,49,8,laguajira,26.700000,2855.86,245.473018,...,44746.452735,105537.118267,85981.834594,169633.602119,3.679528,2,0.0,6.565825,2.974838,0.000000
2,rionegro,1-2018,271.0,2,51,7,antioquia,15.500000,2855.86,343.717029,...,51759.383143,56151.273165,86719.142328,88352.122091,3.679528,12,3.0,6.596442,1.608628,0.046204
3,bogota,1-2018,99767.0,83,7149,426,bogotadc,12.600000,2855.86,23953.819812,...,95874.495457,75289.853012,116471.723062,170536.499037,3.679528,13,0.0,189.975333,37.850345,0.808401
4,piedecuesta,1-2018,28.0,1,66,13,santander,22.523648,2855.86,258.755835,...,59438.480482,63676.087723,86267.558458,96363.989201,3.679528,4,3.0,3.862518,0.983587,0.003875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,inirida,12-2023,9.0,0,5,2,guainia,26.650000,3956.76,28.530179,...,96724.213292,80971.122105,148406.448920,398169.400513,9.276050,0,0.0,3.846943,1.160703,0.001200
4856,lajaguadeibirico,12-2023,49.0,1,7,0,cesar,24.466558,3956.76,417.451225,...,96724.213292,115538.308052,147201.698640,180503.209527,9.276050,1,0.0,2.169964,0.579078,0.007900
4857,quibdo,12-2023,28.0,5,72,7,choco,23.795814,3956.76,131.489163,...,32778.482000,98407.282000,165761.578000,280007.826000,9.276050,2,0.0,7.329747,1.570298,0.017300
4858,mitu,12-2023,1.0,0,3,3,vaupes,23.726524,3956.76,16.948123,...,96724.213292,81662.941349,144525.028720,336723.130284,9.276050,0,0.0,0.990406,0.395778,1.431370


## Variables for which we will make KNN

In [40]:

Base_Turismo_PD.columns[Base_Turismo_PD.isna().any()].tolist()

['Establecimientos de turismo',
 'N Habitaciones',
 'N Camas',
 'Area Urbana',
 'Area Rural',
 'Area Agua']

## Perform KNN to complete missing data for variables ('Tourism Establishments', 'N Rooms', 'N Beds', and the image variables)

In [41]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categoricas = ['Ciudad' , 'Departamento' , 'Mes']
numericas = ['Nmero Extranjeros', 'Homicidios', 'Hurtos',
       'Delitos Sexuales', 'Temperatura', 'Dolar',
       'Pib Ponderado', 'Entradas Extranjeros Zona', 'Distancia a accseos','importancia accesos',
       'Establecimientos de turismo', 'N Habitaciones', 'N Camas', 'Distancia al TOP',
       'Proxy Pobreza', 'Gasto Promedio Diario', 'Gasto Alojamiento Diario',
       'Gasto Transporte Diario', 'Gasto alimetos Diario',
       'Otros Gastos Diario', 'Gasto Promedio Viaje',
       'Gasto  Alojamiento Viaje', 'Gasto Transporte Viaje',
       'Gasto alimetos Viaje', 'Otros Gastos Viaje' , 'Inflacion' , 'Nmero Vias','Eventos' , 'Area Urbana' , 'Area Rural' , 'Area Agua']

# Separar las columnas a imputar
imputar = ['Establecimientos de turismo', 'N Habitaciones', 'N Camas', 'Area Urbana' , 'Area Rural' , 'Area Agua']

# Crear el preprocesador para manejar las columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse=False), categoricas),
        ('num', StandardScaler(), numericas)], remainder='passthrough' )

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', KNNImputer(n_neighbors=5, weights='uniform'))])

KNN1 = pipeline.fit_transform(Base_Turismo_PD)

# Proceso para recuperar los datos sin escala
encoder = preprocessor.named_transformers_['cat']
columnas_encoded = encoder.get_feature_names_out(categoricas)
finales = list(columnas_encoded) + numericas 
base_imputada = pd.DataFrame(KNN1, columns=finales)
scaler = preprocessor.named_transformers_['num']
mean = scaler.mean_
scale = scaler.scale_

for i, var in enumerate(numericas):    
    base_imputada[var] = base_imputada[var] * scale[i] + mean[i]

# Reemplazar las columnas imputadas en el DataFrame original
Base_Turismo_PD1 = Base_Turismo_PD.copy()
Base_Turismo_PD1[imputar] = base_imputada[imputar]


C:\Users\alejo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## We can verify that there is no longer missing data in the DF

In [42]:
# Podemos comprobar que ya no hay datos faltantes en el DF
Base_Turismo_PD1.columns[Base_Turismo_PD1.isna().any()].tolist()

[]

### We finished the database for all years, now we are going to export it and use it for the model and the final descriptive statistics

In [43]:
Base_Turismo_PD1.to_csv('Base Final1.csv', index=False)

### Pablo Reyes